## Analyzing the Impact of TELs on Debt Issues

This Notebook uses the data constructed in [sas2csv](https://github.com/choct155/TELs_debt/blob/master/code/sas2csv.ipynb) and [DebtDataSeries](https://github.com/choct155/TELs_debt/blob/master/code/DebtDataSeries.ipynb) to evaluate the impact of tax and expenditure limitations on debt issues by county.  This Notebook will do the following:

1. Subset to the variables critical to our analysis (**Data Input**);
2. Build specifications that feature a set of debt related dependent variables (**Model Design**);
3. Estimate the relationship between TELs and debt by way of pooled and fixed effect models (**Estimation**).

In [163]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sb
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas.io.data as web

%pylab inline

rcParams['axes.edgecolor']='k'
rcParams['patch.facecolor']='w'
rcParams['axes.facecolor']='w'
rcParams['axes.linewidth']=1

def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## Data Input

Our data is housed in ... the **`data/`** directory.  We are looking for `debt_out.csv` which has aggregate debt issue, institutional, socioeconomic, and spatial information aggregated to the county level.

In [164]:
!ls -l ../data/

total 425632
-rw-r--r-- 1 root     root       165888 Nov 10 17:20 13slsstab1a.xls
-rw-r--r-- 1 root     root        93112 Nov 10 17:20 2013_GFS_debt.xcf
-rw-r--r-- 1 root     root     12226235 Nov 10 17:20 bonds.csv
-rwxr-xr-x 1 root     root     78722993 Nov 15 16:10 costat_mod_vars1940_2010.csv
-rwxr-xr-x 1 root     root      2960775 Nov 15 16:13 cty_coverage.csv
-rw-r--r-- 1 root     root            0 Nov 10 17:20 current_issue_geocode_list.csv
-rw-r--r-- 1 root     root     46828572 Nov 15 22:53 debt_mod.csv
-rw-r--r-- 1 root     root     55709977 Nov 15 18:03 debt_mod_std.csv
-rwxr-xr-x 1 root     root     55859853 Nov 15 16:13 debt_out.csv
-rw-r--r-- 1 root     root     47620501 Nov 10 17:20 debt_ts_pre_fips.csv
-rw-r--r-- 1 root     root     46996855 Nov 15 16:13 debt_w_fips.csv
-rw-r--r-- 1 root     root     56013396 Nov 15 16:13 debt_w_int.csv
-rw-r--r-- 1 root     root      1471282 Nov 15 14:12 Descriptives.csv
-rw-r--r-- 1 root     root       104148 Nov 10 17:20 fips_st_co_0

Let's go ahead and read in the data.

In [165]:
#Read in data
# data_in=pd.read_csv('../data/debt_out.csv')
data_in=pd.read_csv('../data/debt_w_int.csv')

#Generate variable that captures gap between general revenue and direct general expenditure, normalized by exp
data_in['OSRC_GAP']=(data_in['GEN_REV']-data_in['D_GEN_EXP'])/data_in['D_GEN_EXP']

#Generate general revenue per capita variable
data_in['GEN_REV_PC']=data_in['GEN_REV']/data_in['RESPOP']

#Generate tax effort variable
data_in['TAX_EFFORT']=data_in['GEN_REV_PC']/data_in['PC_INC']

#Capture issuer suffixes
issuers=['City, Town Vlg','Co-op Utility','County/Parish','Direct Issuer','District',\
         'Indian Tribe','Local Authority']
purposes=['Development','Education','Electric Power','Environmental Facilities','General Purpose','Healthcare',\
          'Housing','Public Facilities','Transportation','Utilities']

#Define replacement suffixes
issuers_new=['GEN_MUNI','COOP_UTIL','CTY','DIRECT','DISTRICT','TRIBE','LOC_AUTH']
purposes_new=['DEV','EDUC','ELECTRIC','ENVIRON','GEN_PUR','HEALTH','HOUS','PUB_FAC','TRANSPORT','UTIL']

#Capture lists of old and new name pairings
goi=zip(['GO_'+var for var in issuers],['GO_'+var for var in issuers_new])
gop=zip(['GO_'+var for var in purposes],['GO_'+var for var in purposes_new])
rvi=zip(['RV_'+var for var in issuers],['RV_'+var for var in issuers_new])
rvp=zip(['RV_'+var for var in purposes],['RV_'+var for var in purposes_new])

#Build renaming dict
rename_dict=dict(goi+gop+rvi+rvp)

#Rename relevant variables
data_in=data_in.rename(columns=rename_dict)
print 'Before subset:',len(data_in)

#Subset to non-null values of RESPOP and counties in MSAs
data_in=data_in[(data_in['RESPOP'].notnull()) & (data_in['MSA']==1)]
print 'After subset:',len(data_in)
print 'The vast majority of observations lost (all but maybe 100) come in 2011 on (after COSTAT data ends)'

print sorted(data_in.columns),'\n\n',data_in.info()

Before subset: 59669
After subset: 23658
The vast majority of observations lost (all but maybe 100) come in 2011 on (after COSTAT data ends)
['ASMT_L', 'ASMT_L2', 'ASMT_L3', 'BOTH', 'CB_E', 'CB_E2', 'CB_E3', 'CB_E4', 'CB_G', 'CB_G2', 'CFDISC_L', 'CGEXP_L', 'CH_HS_UNT', 'CLEVY_L', 'CLEVY_L2', 'CLEVY_L3', 'CLEVY_L4', 'CRATE_L', 'CRATE_L2', 'CREVU_L', 'CTY_INTEREST', 'DENSITY', 'DIVERSITY', 'D_GEN_EXP', 'EDUC_SERV_EMP_PNFARM', 'EMP_RES', 'FFDISC_L', 'FIPS', 'FIPSST', 'FIPST_N', 'FOOD_SERV_EMP_PNFARM', 'GEN_REV', 'GEN_REV_PC', 'GEXP_L', 'GO', 'GO_COOP_UTIL', 'GO_CTY', 'GO_DEV', 'GO_DIRECT', 'GO_DISTRICT', 'GO_EDUC', 'GO_ELECTRIC', 'GO_ENVIRON', 'GO_GEN_MUNI', 'GO_GEN_PUR', 'GO_HEALTH', 'GO_HOUS', 'GO_LOC_AUTH', 'GO_PUB_FAC', 'GO_TRANSPORT', 'GO_TRIBE', 'GO_UTIL', 'GP_GEXP', 'GP_LEVY', 'GP_LMT', 'GP_RATE', 'GP_REVU', 'HOME_STEAD', 'HOME_STEAD2', 'HOME_STEAD3', 'HSG_UNITS', 'HSG_UNITS_ACS', 'HSLD_PERS', 'IGR_ST', 'LANDAREA', 'LEVY_L', 'LIMITS', 'MANU_EMP_PNFARM', 'MANU_RES', 'MDHOMEVAL', 'ME

In [166]:
data_in[['GEN_REV','D_GEN_EXP','OSRC_GAP']]

,GEN_REV,D_GEN_EXP,OSRC_GAP
5,23020.4,20952.0,0.098721
6,30817.8,26915.6,0.144979
7,32131.6,29090.2,0.104551
8,33445.4,31264.8,0.069746
9,36073.0,35614.0,0.012888
10,40466.0,43001.6,-0.058965
11,44859.0,50389.2,-0.109750
12,53645.0,65164.4,-0.176774
13,58038.0,72552.0,-0.200050
14,63978.2,75520.0,-0.152831


In [167]:
print len(data_in[pd.isnull(data_in).any(axis=1)]),len(data_in)
print len(data_in[pd.isnull(data_in).any(axis=1)])/float(len(data_in))

783 23658
0.0330966269338


Subset to complete cases.

In [168]:
print 'Before subset:',len(data_in)
data_in=data_in[pd.notnull(data_in).all(axis=1)]
print 'After subset:',len(data_in)

Before subset: 23658
After subset: 22875


The set of variables in play appear in the table below:

**DEPENDENT VARIABLES**

Concept|Input Variables
-------|---------------
Per capita GO debt issued|*Variables beginning with GO* & `RES_POP`
Per capita revenue debt issued|*Variables beginning with RV* & `RES_POP`
Ratio of GO to revenue debt issued|*Variables beginning with GO or RV*

**INSTITUTIONAL VARIABLES**

Concept|Input Variables
-------|---------------
Any TEL|`LIMITS`
Non-binding TEL|`TYPE1`
Potentially binding TEL|`TYPE2`
Both `TYPE1` & `TYPE2`|`BOTH`
Years since `TYPE2` enacted|`TYPE2_y`
Overall property tax rate limit|`RATE_L`
Overall assessment limit|`SC_LMT`
Limit applied to general purpose gov|`GP_LMT`
Limit applied to school district|`SC_LMT`

*Note that all limits above can be interacted with primary county status (`PRIMARY`; see spatial table below), in which case we append an `i` to the variable name.*

**SCALE & SUPPLY MEASURES**

Concept|Input Variables
-------|---------------
Population|`RES_POP`
<span style="color:red">Population$^2$</span>|`RES_POP2`
Population density|`DENSITY`
Population growth rate|`POPGROW`
Household size|`PERS_HLD`
Pre-1940 housing stock|`PRE1940`

**DEMAND MEASURES**

Concept|Input Variables
-------|---------------
Population under 17|`PYOUNG`
Private school enrollment|`PVT_SCH`
Population over 65|`POP65`
Per capita income|`PCINC`
Povery rate|`POVERTY`
Average monthly Social Security payments (to recipients)|`PC_SSI`
Per capita income weighted by poverty rate|`DIVERSITY`

**ECONOMIC ACTIVITY**

Concept|Input Variables
-------|---------------
Employment to population ratio|`EMP_RESI`
Manufacturing employment to population ratio|`MANU_RES`
Retail employment to population ratio|`RETL_RES`
Service employment to population ratio|`SERV_RES`

**SPATIAL CHARACTERISTICS**

Concept|Input Variables
-------|---------------
Primary central county in 1974|`PRIMARY`
Co-central county in 1974|`CO_PRIM`
Urban fringe county in 1974|`FRINGE`

Let's grab these in category lists to make them more accessible.

In [169]:
#Capture dependent variables
debt_vars=['GO','RV']
go_vars={'iss':['GO_'+var for var in issuers_new],
         'pur':['GO_'+var for var in purposes_new]}
rv_vars={'iss':['RV_'+var for var in issuers_new],
         'pur':['RV_'+var for var in purposes_new]}

#Capture independent vars
tel_vars={'types':['TYPE1','TYPE2','TYPE2_Y'],
          'either':['LIMITS','BOTH'],
          'hi_res':['RATE_L','ASMT_L','GP_LMT','SC_LMT']}
supply_vars=['RESPOP','DENSITY','POPGROWTH','HSLD_PERS','PRE1940']
demand_vars=['PYOUNG','PVT_SCH','POP65','PC_INC','POVERTY','PC_SSI','DIVERSITY']
economic_vars=['EMP_RES','MANU_RES','RETL_RES','SERV_RES','CTY_INTEREST']
spatial_vars=['PRIMARY','CO_PRIM','FRINGE']
fiscal_vars=['GEN_REV','OSRC_GAP','TAX_EFFORT','IGR_ST','TOT_DEBT_OUTST']

#Capture all modeling variables in a single list
mod_vars=debt_vars+go_vars['iss']+go_vars['pur']+rv_vars['iss']+rv_vars['pur']+tel_vars['types']+\
         tel_vars['either']+tel_vars['hi_res']+supply_vars+demand_vars+economic_vars+spatial_vars+fiscal_vars
    
#For each model variable...
for var in mod_vars:
    #...tell me if it's not in the set
    if var not in data_in.columns:
        print "Is "+var+" in the data set??  Maaaan, we ain't found shit!"
        
#Capture model subset
data=data_in[[var for var in mod_vars if var in data_in.columns]+['Year','FIPS']]

#Generate populations squared
data['RESPOP2']=data['RESPOP']**2

data.head().T

Is PRIMARY in the data set??  Maaaan, we ain't found shit!
Is CO_PRIM in the data set??  Maaaan, we ain't found shit!
Is FRINGE in the data set??  Maaaan, we ain't found shit!


/home/choct155/analysis_tools/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,7,8,9,10,11
GO,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,4.340000e+00
RV,0.000000e+00,2.200000e+00,4.450000e+00,4.545000e+00,1.365000e+00
GO_GEN_MUNI,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,4.340000e+00
GO_COOP_UTIL,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_CTY,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_DIRECT,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_DISTRICT,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_TRIBE,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_LOC_AUTH,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_DEV,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [170]:
print len(data[pd.isnull(data).any(axis=1)]),len(data)
print len(data[pd.isnull(data).any(axis=1)])/float(len(data))

0 22875
0.0


### Accounting for Inflation

The first thing we need to do is adjust all of the dollar figures for inflation.  The relevant variables are in the following list.

In [171]:
#Capture debt variables
debt_vars=[var for var in data.columns if var.startswith('GO')]+\
          [var for var in data.columns if var.startswith('RV')]

#Capture variables to be deflated
defl_vars=debt_vars+['PC_INC','PC_SSI','DIVERSITY']

We need to pull an index for deflation.  To be consistent with the descriptive analysis, we will use the PCE deflator from [FRED](https://research.stlouisfed.org/fred2/).  Specifically, we are using the chained PCE deflator ([PCECTPI](https://research.stlouisfed.org/fred2/)).  We can pull that directly with pandas FRED API and use it to deflate our data.

**UPDATE** We are converting to the state and local government implicit price deflator (`A829RD3A086NBEA`).  The mechanics are the same as with the PCE, so we are leaving the code undisturbed.  Note that the SLD increases faster than the PCE.

In [172]:
#Capture PCE
# pce=web.DataReader("PCECTPI","fred",'1/1/1980','1/1/2015').reset_index()
pce=web.DataReader("A829RD3A086NBEA","fred",'1/1/1980','1/1/2015').reset_index()

#Pull out year
pce['Year']=pce['DATE'].apply(lambda x: x.year)

#Set index
pce.set_index('Year',inplace=True)

#Drop date
pce.pop('DATE')

#Capture average by year
pce=pce.groupby(level='Year').mean()

#Calculate deflators
# pce['defl']=pce['PCECTPI'].apply(lambda x: pce['PCECTPI'].ix[2009]/x)
pce['defl']=pce['A829RD3A086NBEA'].apply(lambda x: pce['A829RD3A086NBEA'].ix[2009]/x)

#Map deflator into data via Year
data['defl']=data['Year'].map(pce['defl'])

print pce.head()

data[['Year','defl']].head(20)

      A829RD3A086NBEA      defl
Year                           
1980           32.583  3.069085
1981           35.824  2.791425
1982           38.012  2.630748
1983           39.700  2.518892
1984           41.407  2.415051


/home/choct155/analysis_tools/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Year,defl
7,1989,1.988941
8,1990,1.894513
9,1992,1.766753
10,1993,1.722683
11,1994,1.678049
12,1996,1.596755
13,1997,1.562598
14,1998,1.531745
15,1999,1.473297
16,2000,1.405284


With the deflators in hand, we can loop through a copy of the data and generate deflated versions of the contents of `defl_vars`.

In [173]:
#Create a data copy to hold deflated values
data_d=data.copy(deep=True)

#For each var to be deflated...
for var in defl_vars:
    #....deflate the variable
    data_d[var]=data_d[var]*data_d['defl']

print data[['GO','RV','Year','defl']].head()    
print '\n',data_d[['GO','RV','Year','defl']].head()

      GO     RV  Year      defl
7   0.00  0.000  1989  1.988941
8   0.00  2.200  1990  1.894513
9   3.00  4.450  1992  1.766753
10  0.00  4.545  1993  1.722683
11  4.34  1.365  1994  1.678049

          GO        RV  Year      defl
7   0.000000  0.000000  1989  1.988941
8   0.000000  4.167930  1990  1.894513
9   5.300260  7.862052  1992  1.766753
10  0.000000  7.829592  1993  1.722683
11  7.282735  2.290537  1994  1.678049


In [174]:
print len(data_d[pd.isnull(data_d).any(axis=1)]),len(data_d)
print len(data_d[pd.isnull(data_d).any(axis=1)])/float(len(data_d))

0 22875
0.0


### Accounting for the Business Cycle

We also need to grab recessionary dates, but some assumptions are required since those are tracked on an monthly basis.  Our imperfect approach will be to treat an year with a majority of months in recession as being a recessionary year.  Since the annual average leaves us with the proportion of recessionary months, a recessionary year is any one in which the average exceeds 0.5.

In [175]:
#Capture recessionary dates
usrec=web.DataReader("USREC","fred",'1/1/1980','1/1/2015').reset_index()

#Pull out year
usrec['Year']=usrec['DATE'].apply(lambda x: x.year)

#Set index
usrec.set_index('Year',inplace=True)

#Drop date
usrec.pop('DATE')

#Capture average by year
usrec=usrec.groupby(level='Year').mean()

#Calculate binary recession variable
usrec['BIN_REC']=np.where(usrec['USREC']>=.5,1,0)

#Map bin_rec into data via Year
data_d['BIN_REC']=data_d['Year'].map(usrec['BIN_REC'])

print usrec.head()

print data_d[['Year','BIN_REC']].head()
print data_d[data_d['Year']==2001][['Year','BIN_REC']].head()

         USREC  BIN_REC
Year                   
1980  0.500000        1
1981  0.416667        0
1982  0.916667        1
1983  0.000000        0
1984  0.000000        0
    Year  BIN_REC
7   1989        0
8   1990        0
9   1992        0
10  1993        0
11  1994        0
     Year  BIN_REC
17   2001        1
48   2001        1
98   2001        1
180  2001        1
255  2001        1


In [176]:
print len(data_d[pd.isnull(data_d).any(axis=1)]),len(data_d)
print len(data_d[pd.isnull(data_d).any(axis=1)])/float(len(data_d))

0 22875
0.0


## Capturing the Prevailing Interest Rate

The prevailing interest rate speaks to the supply of money, and is therefore a useful indicator to understand the macro context.  The coupon rate is missing all values in the raw data, so we will use the 30-year treasury as a proxy.  The inflation-adjusted history is short, so we will manually adjust the nominal rates.  The Federal Reserve has [data on historic market yield of US Treasuries at 30-year constant maturity](http://www.federalreserve.gov/releases/h15/data.htm).  This data has been downloaded and placed in `../data/`.  The [inflation adjustment](https://research.stlouisfed.org/fred2/series/FPCPITOTLZGUSA) to calculate the real rate of interest comes from FRED.

In [177]:
#Read in data
treas30=pd.read_csv('../data/FRB_TREAS_30YR.csv',skiprows=6,header=None,names=['Year','RATE'])

#Replace ND with null values
treas30['RATE']=treas30['RATE'].str.replace('ND','')

#Convert RATE to float
def to_float(x):
    if x!='':
        return float(x)
    else:
        return np.NaN

# treas30['RATE'].apply(lambda x: to_float(x)) NOT WORKING FOR SOME REASON

#Generate float version of RATE
treas30['RATE']=[to_float(val) for val in treas30['RATE']]

#Interpolate for missing values
treas30['RATE']=treas30['RATE'].interpolate()

#Set index
# treas30.set_index('Year',inplace=True)

#Capture CPI
cpi=web.DataReader("FPCPITOTLZGUSA","fred",'1/1/1980','1/1/2015').reset_index()

#Pull out year
cpi['Year']=cpi['DATE'].apply(lambda x: x.year)

#Set index
cpi.set_index('Year',inplace=True)

#Drop date
cpi.pop('DATE')

#Capture average by year
cpi=cpi.groupby(level='Year').mean()

#Join inflation to rate data
treas30['INFLAT']=treas30['Year'].map(cpi['FPCPITOTLZGUSA'])

#Calculate real rate of interest
treas30['REAL_RATE']=treas30['RATE']-treas30['INFLAT']

#Set index
treas30.set_index('Year',inplace=True)

#Map REAL_RATE into data via Year
data_d['REAL_RATE']=data_d['Year'].map(treas30['REAL_RATE'])

#Map INFLAT into data by year
data_d['INFLAT']=data_d['Year'].map(treas30['INFLAT'])

#Create new real interest rate by county
data_d['R_CTY_INT']=data_d['CTY_INTEREST']-data_d['INFLAT']

#Capture difference between prevailing interest rate and county rate
data_d['R_CTY_INT_DIFF']=data_d['REAL_RATE']-data_d['R_CTY_INT']

data_d[['Year','REAL_RATE','CTY_INTEREST','INFLAT','R_CTY_INT','R_CTY_INT_DIFF']].head()

,Year,REAL_RATE,CTY_INTEREST,INFLAT,R_CTY_INT,R_CTY_INT_DIFF
7,1989,3.622997,6.891819,4.827003,2.064816,1.558181
8,1990,3.212044,6.838500,5.397956,1.440544,1.771500
9,1992,4.641180,4.483000,3.028820,1.454180,3.187000
10,1993,3.638343,5.676000,2.951657,2.724343,0.914000
11,1994,4.762558,6.269296,2.607442,3.661854,1.100704


We also need to capture the difference in the prevailing real interest rate.

In [178]:
print len(data_d[pd.isnull(data_d).any(axis=1)]),len(data_d)
print len(data_d[pd.isnull(data_d).any(axis=1)])/float(len(data_d))

0 22875
0.0


In [179]:
#Set index
data_d_tmp=data_d.set_index(['FIPS'])

#Create container for bad counties
bad_cty=[]

#Create container for subsets
dd_subs=[]

#For each county...
for cty in sorted(set(data_d_tmp.index)):
    #...capture subset...
    dd_sub=data_d_tmp.ix[cty]
    #...redefine county...
    dd_sub['FIPS']=cty
    #...define the change in real interest rates...
    if len(shape(dd_sub))>1:
        dd_sub['REAL_RATE_CHANGE']=dd_sub['REAL_RATE'].diff()
        #...and throw the subset back in the list
        dd_subs.append(dd_sub)
    else:
        print dd_sub
        bad_cty.append(cty)
    
#Concatenate back together
data_d=pd.concat(dd_subs)

/home/choct155/analysis_tools/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/choct155/analysis_tools/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


GO                5.588644e+00
RV                0.000000e+00
GO_GEN_MUNI       0.000000e+00
GO_COOP_UTIL      0.000000e+00
GO_CTY            0.000000e+00
GO_DIRECT         0.000000e+00
GO_DISTRICT       5.588644e+00
GO_TRIBE          0.000000e+00
GO_LOC_AUTH       0.000000e+00
GO_DEV            0.000000e+00
GO_EDUC           0.000000e+00
GO_ELECTRIC       0.000000e+00
GO_ENVIRON        0.000000e+00
GO_GEN_PUR        0.000000e+00
GO_HEALTH         0.000000e+00
GO_HOUS           0.000000e+00
GO_PUB_FAC        5.588644e+00
GO_TRANSPORT      0.000000e+00
GO_UTIL           0.000000e+00
RV_GEN_MUNI       0.000000e+00
RV_COOP_UTIL      0.000000e+00
RV_CTY            0.000000e+00
RV_DIRECT         0.000000e+00
RV_DISTRICT       0.000000e+00
RV_TRIBE          0.000000e+00
RV_LOC_AUTH       0.000000e+00
RV_DEV            0.000000e+00
RV_EDUC           0.000000e+00
RV_ELECTRIC       0.000000e+00
RV_ENVIRON        0.000000e+00
                      ...     
DENSITY           4.547754e+01
POPGROWT

/home/choct155/analysis_tools/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [180]:
bad_cty

[22037, 37103, 37115, 48059, 50011, 51157, 54051, 54079, 54105]

In [181]:
print len(data_d[pd.isnull(data_d).any(axis=1)]),len(data_d)
print len(data_d[pd.isnull(data_d).any(axis=1)])/float(len(data_d))

1050 22866
0.0459197061139


In [182]:
data_d=data_d[pd.notnull(data_d).all(axis=1)]
print len(data_d[pd.isnull(data_d).any(axis=1)])

0


### Mapping in State Labels

Just for facilitating the reading of results, state labels would be useful.  Here is a mapping from that FIPS codes.

In [183]:
#Capture state FIPS codes
data_d['FIPSST']=data_d['FIPS'].apply(lambda x: str(x).zfill(5)[:2])

#Map FIPS to state
fips_st_map={'01': 'AL',
             '02': 'AK',
             '04': 'AZ',
             '05': 'AR',
             '06': 'CA',
             '08': 'CO',
             '09': 'CT',
             '10': 'DE',
             '11': 'DC',
             '12': 'FL',
             '13': 'GA',
             '15': 'HI',
             '16': 'ID',
             '17': 'IL',
             '18': 'IN',
             '19': 'IA',
             '20': 'KS',
             '21': 'KY',
             '22': 'LA',
             '23': 'ME',
             '24': 'MD',
             '25': 'MA',
             '26': 'MI',
             '27': 'MN',
             '28': 'MS',
             '29': 'MO',
             '30': 'MT',
             '31': 'NE',
             '32': 'NV',
             '33': 'NH',
             '34': 'NJ',
             '35': 'NM',
             '36': 'NY',
             '37': 'NC',
             '38': 'ND',
             '39': 'OH',
             '40': 'OK',
             '41': 'OR',
             '42': 'PA',
             '44': 'RI',
             '45': 'SC',
             '46': 'SD',
             '47': 'TN',
             '48': 'TX',
             '49': 'UT',
             '50': 'VT',
             '51': 'VA',
             '53': 'WA',
             '54': 'WV',
             '55': 'WI',
             '56': 'WY'}

#Map in state labels
data_d['STATE']=data_d['FIPSST'].map(fips_st_map)

It would also be useful to cluster by BEA region, so we need to map those in as well.

In [184]:
#Capture BEA regions
bea_reg_map={'NENG':['CT','ME','MA','NH','RI','VT'],
             'MEST':['DE','DC','MD','NJ','NY','PA'],
             'GLAK':['IL','IN','MI','OH','WI'],
             'PLNS':['IA','KS','MN','MO','NE','ND','SD'],
             'SEST':['AL','AR','FL','GA','KY','LA','MS','NC','SC','TN','VA','WV'],
             'SWST':['AZ','NM','OK','TX'],
             'RKMT':['CO','ID','MT','UT','WY'],
             'FWST':['AK','CA','HI','NV','OR','WA']}

#Provide integer labels for each region
bea_reg_ints={'NENG':1,
              'MEST':2,
              'GLAK':3,
              'PLNS':4,
              'SEST':5,
              'SWST':6,
              'RKMT':7,
              'FWST':8}

#Create container for tuples (st to reg)
st_reg_tups=[]

#For each region...
for reg in bea_reg_map.keys():
    #...and for each state in the region...
    for st in bea_reg_map[reg]:
        #...throw the st-reg pair in st_reg_tups
        st_reg_tups.append((st,reg))
        
#Capture reverse BEA region dict
bea_st_map=dict(st_reg_tups)

#Define BEA region variable
data_d['BEA']=data_d['STATE'].map(bea_st_map)

In [185]:
#Subset to states
data_d=data_d[~data_d['FIPSST'].isin(['60', '72', '66', '78'])]

#Define integer version of BEA region
data_d['BEA_INT']=data_d['BEA'].map(bea_reg_ints)

data_d[['FIPSST','FIPS','STATE','BEA']].head()

,FIPSST,FIPS,STATE,BEA
FIPS,,,,
1001,01,1001,AL,SEST
1001,01,1001,AL,SEST
1001,01,1001,AL,SEST
1001,01,1001,AL,SEST
1001,01,1001,AL,SEST


It turns out we have issues in here from American Samoa, Guam, Puerto Rico, and the Virgin Islands.  Since they do not have BEA regions, we will be dropping them.  There are only 85 records of this type.

In [186]:
data_d[data_d['BEA'].isnull()][['FIPSST','FIPS','STATE','BEA']]

,FIPSST,FIPS,STATE,BEA
FIPS,,,,


In [187]:
print len(data_d[pd.isnull(data_d).any(axis=1)]),len(data_d)
print len(data_d[pd.isnull(data_d).any(axis=1)])/float(len(data_d))

0 21816
0.0


## Model Design

### Dependent Variables

#### Per Capita Measures

Now that we have our deflated data, we need to generate a few dependent variables:

1. Per Capita GO Debt Issued
2. Per Capita Revenue Debt Issued
3. Proportion of All Debt Issued that is GO

Note also that while we have the totals for all of these items, we also have GO and revenue debt by issuer type and debt purpose.  All of these could serve as dependent variables, so we should generate them systematically.  The first step would be capturing all per capita measures.

In [188]:
data_d[data_d['RESPOP'].isnull()][['Year','FIPS']]

,Year,FIPS
FIPS,,


In [189]:
#Create container for per capita debt variables
pc_debt_vars=[]

#For each debt variable...
for var in debt_vars:
    #...generate per capita versions...
    data_d[var+'_PC']=data_d[var]/data_d['RESPOP']
    data_d[var+'_PC_LN']=data_d[var+'_PC'].apply(lambda x: np.log(x+1))
    #...and store the variable name
#     pc_debt_vars.append(var+'_PC')
    pc_debt_vars.append(var+'_PC_LN')

print pc_debt_vars

['GO_PC_LN', 'GO_GEN_MUNI_PC_LN', 'GO_COOP_UTIL_PC_LN', 'GO_CTY_PC_LN', 'GO_DIRECT_PC_LN', 'GO_DISTRICT_PC_LN', 'GO_TRIBE_PC_LN', 'GO_LOC_AUTH_PC_LN', 'GO_DEV_PC_LN', 'GO_EDUC_PC_LN', 'GO_ELECTRIC_PC_LN', 'GO_ENVIRON_PC_LN', 'GO_GEN_PUR_PC_LN', 'GO_HEALTH_PC_LN', 'GO_HOUS_PC_LN', 'GO_PUB_FAC_PC_LN', 'GO_TRANSPORT_PC_LN', 'GO_UTIL_PC_LN', 'RV_PC_LN', 'RV_GEN_MUNI_PC_LN', 'RV_COOP_UTIL_PC_LN', 'RV_CTY_PC_LN', 'RV_DIRECT_PC_LN', 'RV_DISTRICT_PC_LN', 'RV_TRIBE_PC_LN', 'RV_LOC_AUTH_PC_LN', 'RV_DEV_PC_LN', 'RV_EDUC_PC_LN', 'RV_ELECTRIC_PC_LN', 'RV_ENVIRON_PC_LN', 'RV_GEN_PUR_PC_LN', 'RV_HEALTH_PC_LN', 'RV_HOUS_PC_LN', 'RV_PUB_FAC_PC_LN', 'RV_TRANSPORT_PC_LN', 'RV_UTIL_PC_LN']


In [190]:
len(data_d[pd.isnull(data_d).any(axis=1)])

0

The set of proportional dependents can also be generated systematically.

In [191]:
#Capture absolute debt pairs
debt_var_pairs=zip(debt_vars[:len(debt_vars)/2],debt_vars[len(debt_vars)/2:])

#Create a container for proportional debt variables
prop_debt_vars=[]

#For each pair...
for dp in debt_var_pairs:
    #...generate the GO proportion of total debt (in that area)...
    data_d[dp[0]+'_PROP']=np.where((data_d[dp[0]]+data_d[dp[1]])>0,
                                    data_d[dp[0]]/(data_d[dp[0]]+data_d[dp[1]]),
                                    0)
    data_d[dp[0]+'_PROP_LN']=data_d[dp[0]+'_PROP'].apply(lambda x: np.log(x+1))
    #...and capture the variable name
#     prop_debt_vars.append(dp[0]+'_PROP')
    prop_debt_vars.append(dp[0]+'_PROP_LN')
    
print prop_debt_vars

['GO_PROP_LN', 'GO_GEN_MUNI_PROP_LN', 'GO_COOP_UTIL_PROP_LN', 'GO_CTY_PROP_LN', 'GO_DIRECT_PROP_LN', 'GO_DISTRICT_PROP_LN', 'GO_TRIBE_PROP_LN', 'GO_LOC_AUTH_PROP_LN', 'GO_DEV_PROP_LN', 'GO_EDUC_PROP_LN', 'GO_ELECTRIC_PROP_LN', 'GO_ENVIRON_PROP_LN', 'GO_GEN_PUR_PROP_LN', 'GO_HEALTH_PROP_LN', 'GO_HOUS_PROP_LN', 'GO_PUB_FAC_PROP_LN', 'GO_TRANSPORT_PROP_LN', 'GO_UTIL_PROP_LN']


In [192]:
len(data_d[pd.isnull(data_d).any(axis=1)]),len(data_d)

(0, 21816)

#### Deflation by General Revenue

Another way of looking at debt loads is normalize them by general revenue, which speaks to debt significance among financing sources.  Let's generate a list similar to our per capita set above.

In [193]:
#Create container for revenue deflated debt variables
rd_debt_vars=[]

#For each debt variable...
for var in debt_vars:
    #...generate revenue deflated versions...
    data_d[var+'_RD']=data_d[var]/data_d['GEN_REV']
    data_d[var+'_RD_LN']=data_d[var+'_RD'].apply(lambda x: np.log(x+1))
    #...and store the variable name
#     rd_debt_vars.append(var+'_RD')
    rd_debt_vars.append(var+'_RD_LN')

print rd_debt_vars

['GO_RD_LN', 'GO_GEN_MUNI_RD_LN', 'GO_COOP_UTIL_RD_LN', 'GO_CTY_RD_LN', 'GO_DIRECT_RD_LN', 'GO_DISTRICT_RD_LN', 'GO_TRIBE_RD_LN', 'GO_LOC_AUTH_RD_LN', 'GO_DEV_RD_LN', 'GO_EDUC_RD_LN', 'GO_ELECTRIC_RD_LN', 'GO_ENVIRON_RD_LN', 'GO_GEN_PUR_RD_LN', 'GO_HEALTH_RD_LN', 'GO_HOUS_RD_LN', 'GO_PUB_FAC_RD_LN', 'GO_TRANSPORT_RD_LN', 'GO_UTIL_RD_LN', 'RV_RD_LN', 'RV_GEN_MUNI_RD_LN', 'RV_COOP_UTIL_RD_LN', 'RV_CTY_RD_LN', 'RV_DIRECT_RD_LN', 'RV_DISTRICT_RD_LN', 'RV_TRIBE_RD_LN', 'RV_LOC_AUTH_RD_LN', 'RV_DEV_RD_LN', 'RV_EDUC_RD_LN', 'RV_ELECTRIC_RD_LN', 'RV_ENVIRON_RD_LN', 'RV_GEN_PUR_RD_LN', 'RV_HEALTH_RD_LN', 'RV_HOUS_RD_LN', 'RV_PUB_FAC_RD_LN', 'RV_TRANSPORT_RD_LN', 'RV_UTIL_RD_LN']


In [194]:
len(data_d[pd.isnull(data_d).any(axis=1)]),len(data_d)

(0, 21816)

In [195]:
print 'Before subset:',len(data_d)
data_d=data_d[pd.notnull(data_d).all(axis=1)]
print 'After subset:',len(data_d)

Before subset: 21816
After subset: 21816


### Debt Proportions

It may also prove useful to capture the proportions of debt by issuer type or purpose.  The idea here would be to explore the impact of debt classifications on the issuance of debt.  (In other words, we are talking about these proportions going on the independent side of the equation.)  Proportionality allows us to separate these regressors from the absolute debt figures on the dependent side.

For this purpose, we will actually focus on total debt to limit model complexity.

In [196]:
#Calculate total debt
data_d['TOT_DEBT']=data_d['GO']+data_d['RV']

#Calculate total debt per capita
data_d['TOT_DEBT_PC']=data_d['TOT_DEBT']/data_d['RESPOP']
data_d['TOT_DEBT_PC_LN']=data_d['TOT_DEBT_PC'].apply(lambda x: np.log(x+1))

#Create container for total debt variables
tot_debt_vars=['TOT_DEBT']

#For each of the remaining debt pairs...
for dp in debt_var_pairs[1:]:
    #...calculate the total...
    data_d['TOT'+dp[0][2:]]=data_d[dp[0]]+data_d[dp[1]]
    #...and capture the var name
    tot_debt_vars.append('TOT'+dp[0][2:])
    
#Create a container for proportions of total debt
prop_of_tot_debt_vars=[]

#For each of the total debt subsets...
for td in tot_debt_vars[1:]:
    #...calculate the proportion of total debt...
    data_d['TPROP'+td[3:]]=np.where(data_d['TOT_DEBT']>0,
                                    data_d[td]/data_d['TOT_DEBT'],
                                    0)
    #...and capture the name
    prop_of_tot_debt_vars.append('TPROP'+td[3:])
    
#Split out type from purpose
tprop_vars={'ISSUER':prop_of_tot_debt_vars[:10],
            'PURPOSE':prop_of_tot_debt_vars[10:]}
tprop_vars

{'ISSUER': ['TPROP_GEN_MUNI',
  'TPROP_COOP_UTIL',
  'TPROP_CTY',
  'TPROP_DIRECT',
  'TPROP_DISTRICT',
  'TPROP_TRIBE',
  'TPROP_LOC_AUTH',
  'TPROP_DEV',
  'TPROP_EDUC',
  'TPROP_ELECTRIC'],
 'PURPOSE': ['TPROP_ENVIRON',
  'TPROP_GEN_PUR',
  'TPROP_HEALTH',
  'TPROP_HOUS',
  'TPROP_PUB_FAC',
  'TPROP_TRANSPORT',
  'TPROP_UTIL']}

In [197]:
len(data_d[pd.isnull(data_d).any(axis=1)])

0

### Indepenent Variables

There are a few regressor lists we should construct, which vary largely with respect to how TELs are represented.

In [198]:
#Capture lists of regressors
reg1=['TYPE1','TYPE2','TYPE2_Y','RESPOP','RESPOP2','DENSITY','POPGROWTH','HSLD_PERS','PRE1940',\
      'PYOUNG','PVT_SCH','POP65','PC_INC','POVERTY','PC_SSI','DIVERSITY','EMP_RES','MANU_RES',\
      'RETL_RES','SERV_RES','BIN_REC','OSRC_GAP','REAL_RATE','R_CTY_INT_DIFF','GEN_REV','TAX_EFFORT',\
      'IGR_ST','REAL_RATE_CHANGE','TOT_DEBT_OUTST']
reg2=['LIMITS','BOTH']+reg1[3:]
reg3=['RATE_L','ASMT_L','GP_LMT','SC_LMT']+reg1[3:]

#Capture in dict
reg_dict={'TYPE':reg1,
          'AGG':reg2,
          'HIRES':reg3}

### Fixed Effects

We have four fixed effect options which will be included in this analysis: `pooled`, `year fixed effects`, `state fixed effects`, and `both`.  These can be appended to each specification via simple extensions.  These extensions can be captured in a dictionary for easy access.

In [199]:
fe_dict={'POOLED':[],
         'YEAR':['C(Year)'],
         'STATE':['C(FIPSST)'],
         'BOTH':['C(Year)+C(FIPSST)']}

### Specification Builds

We now have host of moving parts on the specification side of the equation.  There are 63 possible dependent variables for the debt type breakouts (`GO` vs `RV`) and one total debt dependent for use in two models evaluating the debt composition impact.  Couple this with three regressor sets and four fixed effect options, and we have a whole mess of specifications (780).  

An orderly collection of formulas would be most useful, to say the least.  It perhaps goes without saying here, but not all of these models will receive the same level of scrutiny.  We will focus on a subset in any detailed discussion of results.  However, running all of these models gives us a chance to test variance in our TEL measures under a large variety of specifications.  Each group of TEL variables will have 260 estimates associated with it.

We will house our specifications in a hierarchical dictionary.  The first level keys will capture four main groups:

1. **Per Capita GO models** (`PC_GO`) are defined by their dependent variables (all per capita GO variables) at the next level down.  Only the simple regressor lists (without debt composition by issuer type or purpose) will be used.
2. **Per Capita RV models** (`PC_RV`) are defined by their dependent variables (all per capita RV variables) at the next level down.  Only the simple regressor lists (without debt composition by issuer type or purpose) will be used.
3. **Proportional Models** (`PROP`) both use total debt issued as the dependent.  The simple regressor lists are augmented by debt composition.  Whether this composition is split by issuer type (`ISSUER`) or debt purpose (`PURPOSE`) defines these models at the next level.

The second level being captured by the definitions within each group of the first level, the third level rotates across each regressor list captured in `reg_dict`.  The fourth level captures the fixed effect combinations.

The first level groups differ strongly due to the proportional group, so we will build each group separetely, and then manually combine them in the master dictionary.

In [200]:
[var for var in pc_debt_vars if (var.startswith('GO'))]

['GO_PC_LN',
 'GO_GEN_MUNI_PC_LN',
 'GO_COOP_UTIL_PC_LN',
 'GO_CTY_PC_LN',
 'GO_DIRECT_PC_LN',
 'GO_DISTRICT_PC_LN',
 'GO_TRIBE_PC_LN',
 'GO_LOC_AUTH_PC_LN',
 'GO_DEV_PC_LN',
 'GO_EDUC_PC_LN',
 'GO_ELECTRIC_PC_LN',
 'GO_ENVIRON_PC_LN',
 'GO_GEN_PUR_PC_LN',
 'GO_HEALTH_PC_LN',
 'GO_HOUS_PC_LN',
 'GO_PUB_FAC_PC_LN',
 'GO_TRANSPORT_PC_LN',
 'GO_UTIL_PC_LN']

In [201]:
## DEFINE FUNCTION TO CAPTURE PER CAPITA SPECS ##
def spec_dict_build(var_list):
    #Create dictionary to hold outgoing specs
    out_dict={}
    #For each per capita dependent...
    for dep in var_list:
        #...create a temp dict to hold specs for all three regressor groups...
        tmp_reg_dict={}
        #...and for each regressor set...
        for rd in reg_dict.keys():
            #...create a temp dict to hold specs for all four fixed effect types...
            tmp_spec_dict={}
            #...and for each FE type...
            for fe in fe_dict.keys():
                #...build the spec and throw it in tmp_spec_dict...
                tmp_spec_dict.update({fe:dep+'~'+'+'.join(reg_dict[rd]+fe_dict[fe])})
            #...once tmp_spec_dict is full, throw it in tmp_reg_dict...
            tmp_reg_dict.update({rd:tmp_spec_dict})
        #...and once tmp_reg_dict is full, throw it in out_dict
#         if dep[3:-3]=='':
#             out_dict.update({'Total':tmp_reg_dict})
#         else:
#             out_dict.update({dep[3:-3]:tmp_reg_dict})
        if dep[3:-6]=='':
            out_dict.update({'Total':tmp_reg_dict})
        else:
            out_dict.update({dep[3:-6]:tmp_reg_dict})
    return out_dict

## CAPTURE DICTIONARY FOR PER CAPITA GO SPECS ##
go_spec_dict=spec_dict_build([var for var in pc_debt_vars if var.startswith('GO')])

## CAPTURE DICTIONARY FOR PER CAPITA GO SPECS ##
rv_spec_dict=spec_dict_build([var for var in pc_debt_vars if var.startswith('RV')])

## CAPTURE DICTIONARY FOR PROPORTIONAL SPECS ##

#Create dictionary to hold prop specs
prop_dict={}

#For each prop key...
for key in ['ISSUER','PURPOSE']:
    #...create a temp dict to hold specs for all three regressor groups...
    tmp_reg_dict={}
    #...and for each regressor set...
    for rd in reg_dict.keys():
        #...create a temp dict to hold specs for all four fixed effect types...
        tmp_spec_dict={}
        #...and for each FE type...
        for fe in fe_dict.keys():
            #...build the spec and throw it in tmp_spec_dict...
            tmp_spec_dict.update({fe:'TOT_DEBT_PC'+'~'+'+'.join(reg_dict[rd]+tprop_vars[key]+fe_dict[fe])})
        #...once tmp_spec_dict is full, throw it in tmp_reg_dict...
        tmp_reg_dict.update({rd:tmp_spec_dict})
    #...and once tmp_reg_dict is full, throw it in prop_dict
    prop_dict.update({key:tmp_reg_dict})
    
## CAPTURE DICTIONARY FOR REVENUE DEFLATED GO SPECS ##
go_rd_spec_dict=spec_dict_build([var for var in rd_debt_vars if var.startswith('GO')])

## CAPTURE DICTIONARY FOR REVENUE DEFLATED GO SPECS ##
rv_rd_spec_dict=spec_dict_build([var for var in rd_debt_vars if var.startswith('RV')])
    
## CAPTURE FIRST LEVEL DICTS IN SPECIFICATIONS DICT ##
spec_dict={'PC_GO':go_spec_dict,
           'PC_RV':rv_spec_dict,
           'PROP':prop_dict,
           'RD_GO':go_rd_spec_dict,
           'RD_RV':rv_rd_spec_dict}

Just to provide a sense of how this works, let's say we want to regress revenue debt per capita for housing on TELs split by type with year fixed effects, we can call that spec with the following:

In [202]:
spec_dict['PC_RV']['Total']['TYPE']['YEAR']

'RV_PC_LN~TYPE1+TYPE2+TYPE2_Y+RESPOP+RESPOP2+DENSITY+POPGROWTH+HSLD_PERS+PRE1940+PYOUNG+PVT_SCH+POP65+PC_INC+POVERTY+PC_SSI+DIVERSITY+EMP_RES+MANU_RES+RETL_RES+SERV_RES+BIN_REC+OSRC_GAP+REAL_RATE+R_CTY_INT_DIFF+GEN_REV+TAX_EFFORT+IGR_ST+REAL_RATE_CHANGE+TOT_DEBT_OUTST+C(Year)'

If I wanted to evaluate the impact of aggregate TEL measures and debt composition by issuer type on total debt per capita, using state fixed effects, this call would work:

In [203]:
spec_dict['PROP']['ISSUER']['AGG']['STATE']

'TOT_DEBT_PC~LIMITS+BOTH+RESPOP+RESPOP2+DENSITY+POPGROWTH+HSLD_PERS+PRE1940+PYOUNG+PVT_SCH+POP65+PC_INC+POVERTY+PC_SSI+DIVERSITY+EMP_RES+MANU_RES+RETL_RES+SERV_RES+BIN_REC+OSRC_GAP+REAL_RATE+R_CTY_INT_DIFF+GEN_REV+TAX_EFFORT+IGR_ST+REAL_RATE_CHANGE+TOT_DEBT_OUTST+TPROP_GEN_MUNI+TPROP_COOP_UTIL+TPROP_CTY+TPROP_DIRECT+TPROP_DISTRICT+TPROP_TRIBE+TPROP_LOC_AUTH+TPROP_DEV+TPROP_EDUC+TPROP_ELECTRIC+C(FIPSST)'

## Estimation

In [204]:
data_d.to_csv('../data/debt_mod.csv')

Now we are in a position to estimate some models.  We will first estimate some top line models, and then move into the analysis of all specs.  The top line models are as follows:

1. Total GO debt per capita on TELs by type with year and state fixed effects;
2. Total RV debt per capita on TELs by type with year and state fixed effects;
3. Total debt per capita on TELs by type and debt composition by issuer type, with year and state fixed effects;
4. Total debt per capita on TELs by type and debt composition by purpose, with year and state fixed effects.

In [205]:
print test_spec

GO_PC~TYPE1+TYPE2+TYPE2_Y+RESPOP+RESPOP2+DENSITY+POPGROWTH+HSLD_PERS+PRE1940+PYOUNG+PVT_SCH+POP65+PC_INC+POVERTY+PC_SSI+DIVERSITY+EMP_RES+MANU_RES+RETL_RES+SERV_RES+BIN_REC+OSRC_GAP+REAL_RATE+R_CTY_INT_DIFF+GEN_REV+TAX_EFFORT+IGR_ST+REAL_RATE_CHANGE+TOT_DEBT_OUTST+C(Year)+C(FIPSST)


In [206]:
test_spec='GO_PC~TYPE1+TYPE2+TYPE2_Y+RESPOP+RESPOP2+DENSITY+POPGROWTH+HSLD_PERS+PRE1940+PYOUNG+PVT_SCH+POP65+PC_INC+POVERTY+PC_SSI+DIVERSITY+EMP_RES+MANU_RES+RETL_RES+SERV_RES+BIN_REC+OSRC_GAP+REAL_RATE+R_CTY_INT_DIFF+GEN_REV+TAX_EFFORT+IGR_ST+REAL_RATE_CHANGE+TOT_DEBT_OUTST+C(Year)+C(FIPSST)'
smf.wls(formula=test_spec,data=data_d).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  GO_PC   R-squared:                       0.118
Model:                            WLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                -1.495e+15
Date:                Sun, 15 Nov 2015   Prob (F-statistic):               1.00
Time:                        23:25:31   Log-Likelihood:             1.3192e+05
No. Observations:               21816   AIC:                        -2.637e+05
Df Residuals:                   21739   BIC:                        -2.631e+05
Df Model:                          76                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0002      0.000     -1.232      0.218        -0.001     0.000
C(Year)[T.1986]  -1.056e-05   1.94e-05     -0.544      0.587     -4.86e-05  2.75e-05
C(Year)[T.1987]  -8.868e-05    1.5e-05     -5.907      0.000        -0.000 -5.93e-05
C(Year)[T.1988]     -0.0003   5.17e-05     -6.402      0.000        -0.000    -0.000
C(Year)[T.1989]     -0.0003    3.4e-05     -9.426      0.000        -0.000    -0.000
C(Year)[T.1990]  -5.649e-05   1.18e-05     -4.799      0.000     -7.96e-05 -3.34e-05
C(Year)[T.1991]   -1.87e-05   3.19e-05     -0.587      0.557     -8.11e-05  4.37e-05
C(Year)[T.1992]   4.765e-05   5.55e-05      0.859      0.391     -6.11e-05     0.000
C(Year)[T.1993]      0.0001    2.6e-05      4.328      0.000      6.15e-05     0.000
C(Year)[T.1994]  -3.987e-05   6.39e-05     -0.624      0.533        -0.000  8.54e-05
C(Year)[T.1995]  -5.625e-05   3.89e-05     -1.444      0.149        -0.000  2.01e-05
C(Year)[T.1996]  -1.068e-05   3.67e-05     -0.291      0.771     -8.26e-05  6.12e-05
C(Year)[T.1997]  -1.888e-05   6.21e-05     -0.304      0.761        -0.000     0.000
C(Year)[T.1998]   6.612e-05   4.78e-05      1.382      0.167     -2.77e-05     0.000
C(Year)[T.1999]  -7.912e-06   4.31e-05     -0.184      0.854     -9.24e-05  7.66e-05
C(Year)[T.2000]  -2.701e-05   1.67e-05     -1.620      0.105     -5.97e-05  5.67e-06
C(Year)[T.2001]   6.855e-05   2.66e-05      2.574      0.010      1.64e-05     0.000
C(Year)[T.2002]   7.233e-05   7.96e-05      0.908      0.364     -8.37e-05     0.000
C(Year)[T.2003]   6.608e-05    4.7e-05      1.407      0.159      -2.6e-05     0.000
C(Year)[T.2004]     -0.0003   4.63e-05     -6.661      0.000        -0.000    -0.000
C(Year)[T.2005]   8.281e-05   4.45e-05      1.862      0.063     -4.37e-06     0.000
C(Year)[T.2006]  -2.712e-05   3.32e-05     -0.818      0.414     -9.21e-05  3.79e-05
C(Year)[T.2007]   3.307e-05   5.14e-05      0.643      0.520     -6.77e-05     0.000
C(Year)[T.2008]    -4.5e-05   6.69e-05     -0.672      0.501        -0.000  8.62e-05
C(Year)[T.2009]  -3.435e-05      0.000     -0.324      0.746        -0.000     0.000
C(Year)[T.2010]  -1.224e-05   3.08e-05     -0.397      0.691     -7.26e-05  4.81e-05
C(FIPSST)[T.04]   6.511e-06   6.05e-05      0.108      0.914        -0.000     0.000
C(FIPSST)[T.05]   5.981e-05   1.85e-05      3.236      0.001      2.36e-05   9.6e-05
C(FIPSST)[T.06]  -8.928e-06   7.41e-05     -0.120      0.904        -0.000     0.000
C(FIPSST)[T.08]   4.207e-05   9.74e-05      0.432      0.666        -0.000     0.000
C(FIPSST)[T.09]      0.0002   8.63e-05      2.597      0.009       5.5e-05     0.000
C(FIPSST)[T.10]  -3.663e-05    4.4e-05     -0.832      0.406        -0.000  4.97e-05
C(FIPSST)[T.12]   -9.06e-05   3.15e-05     -2.874      0.004        -0.000 -2.88e-05
C(FIPSST)[T.13]   5.522

In [207]:
smf.wls(formula=test_spec,data=data_d,weights=data_d['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  GO_PC   R-squared:                       0.171
Model:                            WLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                 2.636e+15
Date:                Sun, 15 Nov 2015   Prob (F-statistic):           7.98e-54
Time:                        23:25:31   Log-Likelihood:             1.2254e+05
No. Observations:               21816   AIC:                        -2.450e+05
Df Residuals:                   21771   BIC:                        -2.446e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0009      0.000     -1.963      0.050        -0.002 -1.18e-06
C(Year)[T.1986]  -1.492e-06   5.29e-05     -0.028      0.978        -0.000     0.000
C(Year)[T.1987]     -0.0002      3e-05     -6.231      0.000        -0.000    -0.000
C(Year)[T.1988]     -0.0004   5.77e-05     -7.536      0.000        -0.001    -0.000
C(Year)[T.1989]     -0.0005   5.66e-05     -8.499      0.000        -0.001    -0.000
C(Year)[T.1990]     -0.0001   2.55e-05     -5.283      0.000        -0.000 -8.48e-05
C(Year)[T.1991]   6.451e-06   5.27e-05      0.122      0.903     -9.69e-05     0.000
C(Year)[T.1992]   7.492e-05   5.88e-05      1.273      0.203     -4.04e-05     0.000
C(Year)[T.1993]      0.0001   6.29e-05      1.756      0.079     -1.28e-05     0.000
C(Year)[T.1994]   5.222e-06   6.62e-05      0.079      0.937        -0.000     0.000
C(Year)[T.1995]  -6.877e-05   4.34e-05     -1.583      0.113        -0.000  1.64e-05
C(Year)[T.1996]  -2.648e-05   4.27e-05     -0.621      0.535        -0.000  5.71e-05
C(Year)[T.1997]   8.791e-06   6.59e-05      0.133      0.894        -0.000     0.000
C(Year)[T.1998]   4.233e-05    3.9e-05      1.086      0.277      -3.4e-05     0.000
C(Year)[T.1999]  -3.023e-05   3.48e-05     -0.869      0.385     -9.84e-05  3.79e-05
C(Year)[T.2000]     -0.0001   6.06e-05     -1.725      0.085        -0.000  1.42e-05
C(Year)[T.2001]   5.374e-05   1.76e-05      3.060      0.002      1.93e-05  8.82e-05
C(Year)[T.2002]   7.978e-05   4.15e-05      1.922      0.055     -1.57e-06     0.000
C(Year)[T.2003]   4.388e-05   5.42e-05      0.810      0.418     -6.23e-05     0.000
C(Year)[T.2004]     -0.0005   9.35e-05     -4.889      0.000        -0.001    -0.000
C(Year)[T.2005]   9.786e-06   7.88e-05      0.124      0.901        -0.000     0.000
C(Year)[T.2006]   -3.76e-05    5.7e-05     -0.660      0.509        -0.000  7.41e-05
C(Year)[T.2007]  -3.598e-06   8.88e-05     -0.040      0.968        -0.000     0.000
C(Year)[T.2008]     -0.0001   5.39e-05     -2.240      0.025        -0.000 -1.51e-05
C(Year)[T.2009]   3.657e-05   6.36e-05      0.575      0.565     -8.81e-05     0.000
C(Year)[T.2010]   -5.26e-05    4.7e-05     -1.119      0.263        -0.000  3.95e-05
C(FIPSST)[T.04]  -4.868e-05   8.23e-05     -0.591      0.554        -0.000     0.000
C(FIPSST)[T.05]   8.307e-05   8.97e-05      0.926      0.355     -9.28e-05     0.000
C(FIPSST)[T.06]  -2.564e-07   5.21e-05     -0.005      0.996        -0.000     0.000
C(FIPSST)[T.08]   4.495e-05   7.98e-05      0.563      0.573        -0.000     0.000
C(FIPSST)[T.09]      0.0003   8.03e-05      3.806      0.000         0.000     0.000
C(FIPSST)[T.10]   7.339e-06   5.13e-05      0.143      0.886     -9.32e-05     0.000
C(FIPSST)[T.12]     -0.0001   7.07e-05     -2.057      0.040        -0.000 -6.89e-06
C(FIPSST)[T.13]   2.219

In [208]:
smf.wls(formula=topline_specs['GO'],data=data_d,weights=data_d['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               GO_PC_LN   R-squared:                       0.175
Model:                            WLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                -1.443e+14
Date:                Sun, 15 Nov 2015   Prob (F-statistic):               1.00
Time:                        23:25:32   Log-Likelihood:             1.2284e+05
No. Observations:               21816   AIC:                        -2.456e+05
Df Residuals:                   21771   BIC:                        -2.452e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0008      0.000     -1.950      0.051        -0.002  4.34e-06
C(Year)[T.1986]  -1.285e-06   5.27e-05     -0.024      0.981        -0.000     0.000
C(Year)[T.1987]     -0.0002   2.96e-05     -6.287      0.000        -0.000    -0.000
C(Year)[T.1988]     -0.0004   5.76e-05     -7.546      0.000        -0.001    -0.000
C(Year)[T.1989]     -0.0005   5.65e-05     -8.499      0.000        -0.001    -0.000
C(Year)[T.1990]     -0.0001   2.52e-05     -5.318      0.000        -0.000 -8.47e-05
C(Year)[T.1991]    7.03e-06   5.26e-05      0.134      0.894     -9.61e-05     0.000
C(Year)[T.1992]   7.542e-05   5.84e-05      1.292      0.197      -3.9e-05     0.000
C(Year)[T.1993]      0.0001   6.26e-05      1.769      0.077     -1.19e-05     0.000
C(Year)[T.1994]   5.963e-06   6.58e-05      0.091      0.928        -0.000     0.000
C(Year)[T.1995]    -6.8e-05   4.29e-05     -1.584      0.113        -0.000  1.61e-05
C(Year)[T.1996]  -2.568e-05   4.22e-05     -0.609      0.542        -0.000  5.69e-05
C(Year)[T.1997]   9.626e-06   6.54e-05      0.147      0.883        -0.000     0.000
C(Year)[T.1998]   4.315e-05   3.83e-05      1.126      0.260      -3.2e-05     0.000
C(Year)[T.1999]  -2.928e-05   3.42e-05     -0.857      0.391     -9.62e-05  3.77e-05
C(Year)[T.2000]     -0.0001   6.04e-05     -1.712      0.087        -0.000  1.49e-05
C(Year)[T.2001]   5.381e-05   1.75e-05      3.068      0.002      1.94e-05  8.82e-05
C(Year)[T.2002]   8.078e-05   4.09e-05      1.977      0.048      6.86e-07     0.000
C(Year)[T.2003]   4.492e-05   5.35e-05      0.839      0.401        -6e-05     0.000
C(Year)[T.2004]     -0.0005   9.34e-05     -4.880      0.000        -0.001    -0.000
C(Year)[T.2005]   1.088e-05    7.9e-05      0.138      0.890        -0.000     0.000
C(Year)[T.2006]  -3.646e-05    5.7e-05     -0.639      0.523        -0.000  7.53e-05
C(Year)[T.2007]  -3.951e-06   8.82e-05     -0.045      0.964        -0.000     0.000
C(Year)[T.2008]     -0.0001    5.4e-05     -2.226      0.026        -0.000 -1.44e-05
C(Year)[T.2009]   3.706e-05   6.33e-05      0.586      0.558      -8.7e-05     0.000
C(Year)[T.2010]  -5.148e-05   4.66e-05     -1.105      0.269        -0.000  3.98e-05
C(FIPSST)[T.04]  -4.887e-05   8.21e-05     -0.595      0.552        -0.000     0.000
C(FIPSST)[T.05]   8.211e-05    8.9e-05      0.922      0.356     -9.24e-05     0.000
C(FIPSST)[T.06]  -2.786e-08   5.16e-05     -0.001      1.000        -0.000     0.000
C(FIPSST)[T.08]   4.551e-05   7.87e-05      0.579      0.563        -0.000     0.000
C(FIPSST)[T.09]      0.0003   7.98e-05      3.822      0.000         0.000     0.000
C(FIPSST)[T.10]   6.829e-06   5.12e-05      0.133      0.894     -9.35e-05     0.000
C(FIPSST)[T.12]     -0.0001   7.03e-05     -2.076      0.038        -0.000 -8.18e-06
C(FIPSST)[T.13]   2.126

In [209]:
#Capture topline specs
topline_specs={'GO':spec_dict['PC_GO']['Total']['TYPE']['BOTH'],
               'RV':spec_dict['PC_RV']['Total']['TYPE']['BOTH'],
               'ISSUER':spec_dict['PROP']['ISSUER']['TYPE']['BOTH'],
               'PURPOSE':spec_dict['PROP']['PURPOSE']['TYPE']['BOTH'],
               'RD_GO':spec_dict['RD_GO']['Total']['TYPE']['BOTH'],
               'RD_RV':spec_dict['RD_RV']['Total']['TYPE']['BOTH'],}

#Estimate each model
# topline_mods={'GO':smf.ols(formula=topline_specs['GO'],data=data_d).fit(cov_type='cluster',
#                                                                        cov_kwds={'groups':data_d['BEA_INT']})}
# topline_mods={'GO':smf.ols(formula=topline_specs['GO'],data=data_d).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}),
#               'RV':smf.ols(formula=topline_specs['RV'],data=data_d).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}),
#               'ISSUER':smf.ols(formula=topline_specs['ISSUER'],data=data_d).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}),
#               'PURPOSE':smf.ols(formula=topline_specs['PURPOSE'],data=data_d).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}),
#               'RD_GO':smf.ols(formula=topline_specs['RD_GO'],data=data_d).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}),
#               'RD_RV':smf.ols(formula=topline_specs['RD_RV'],data=data_d).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']})}

topline_mods={'GO':smf.wls(formula=topline_specs['GO'],data=data_d,weights=data_d['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}),
              'RV':smf.wls(formula=topline_specs['RV'],data=data_d,weights=data_d['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}),
              'ISSUER':smf.wls(formula=topline_specs['ISSUER'],data=data_d,weights=data_d['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}),
              'PURPOSE':smf.wls(formula=topline_specs['PURPOSE'],data=data_d,weights=data_d['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}),
              'RD_GO':smf.wls(formula=topline_specs['RD_GO'],data=data_d,weights=data_d['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']}),
              'RD_RV':smf.wls(formula=topline_specs['RD_RV'],data=data_d,weights=data_d['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_d['BEA_INT']})}

topline_mods['GO'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               GO_PC_LN   R-squared:                       0.175
Model:                            WLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                -1.443e+14
Date:                Sun, 15 Nov 2015   Prob (F-statistic):               1.00
Time:                        23:25:34   Log-Likelihood:             1.2284e+05
No. Observations:               21816   AIC:                        -2.456e+05
Df Residuals:                   21771   BIC:                        -2.452e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0008      0.000     -1.950      0.051        -0.002  4.34e-06
C(Year)[T.1986]  -1.285e-06   5.27e-05     -0.024      0.981        -0.000     0.000
C(Year)[T.1987]     -0.0002   2.96e-05     -6.287      0.000        -0.000    -0.000
C(Year)[T.1988]     -0.0004   5.76e-05     -7.546      0.000        -0.001    -0.000
C(Year)[T.1989]     -0.0005   5.65e-05     -8.499      0.000        -0.001    -0.000
C(Year)[T.1990]     -0.0001   2.52e-05     -5.318      0.000        -0.000 -8.47e-05
C(Year)[T.1991]    7.03e-06   5.26e-05      0.134      0.894     -9.61e-05     0.000
C(Year)[T.1992]   7.542e-05   5.84e-05      1.292      0.197      -3.9e-05     0.000
C(Year)[T.1993]      0.0001   6.26e-05      1.769      0.077     -1.19e-05     0.000
C(Year)[T.1994]   5.963e-06   6.58e-05      0.091      0.928        -0.000     0.000
C(Year)[T.1995]    -6.8e-05   4.29e-05     -1.584      0.113        -0.000  1.61e-05
C(Year)[T.1996]  -2.568e-05   4.22e-05     -0.609      0.542        -0.000  5.69e-05
C(Year)[T.1997]   9.626e-06   6.54e-05      0.147      0.883        -0.000     0.000
C(Year)[T.1998]   4.315e-05   3.83e-05      1.126      0.260      -3.2e-05     0.000
C(Year)[T.1999]  -2.928e-05   3.42e-05     -0.857      0.391     -9.62e-05  3.77e-05
C(Year)[T.2000]     -0.0001   6.04e-05     -1.712      0.087        -0.000  1.49e-05
C(Year)[T.2001]   5.381e-05   1.75e-05      3.068      0.002      1.94e-05  8.82e-05
C(Year)[T.2002]   8.078e-05   4.09e-05      1.977      0.048      6.86e-07     0.000
C(Year)[T.2003]   4.492e-05   5.35e-05      0.839      0.401        -6e-05     0.000
C(Year)[T.2004]     -0.0005   9.34e-05     -4.880      0.000        -0.001    -0.000
C(Year)[T.2005]   1.088e-05    7.9e-05      0.138      0.890        -0.000     0.000
C(Year)[T.2006]  -3.646e-05    5.7e-05     -0.639      0.523        -0.000  7.53e-05
C(Year)[T.2007]  -3.951e-06   8.82e-05     -0.045      0.964        -0.000     0.000
C(Year)[T.2008]     -0.0001    5.4e-05     -2.226      0.026        -0.000 -1.44e-05
C(Year)[T.2009]   3.706e-05   6.33e-05      0.586      0.558      -8.7e-05     0.000
C(Year)[T.2010]  -5.148e-05   4.66e-05     -1.105      0.269        -0.000  3.98e-05
C(FIPSST)[T.04]  -4.887e-05   8.21e-05     -0.595      0.552        -0.000     0.000
C(FIPSST)[T.05]   8.211e-05    8.9e-05      0.922      0.356     -9.24e-05     0.000
C(FIPSST)[T.06]  -2.786e-08   5.16e-05     -0.001      1.000        -0.000     0.000
C(FIPSST)[T.08]   4.551e-05   7.87e-05      0.579      0.563        -0.000     0.000
C(FIPSST)[T.09]      0.0003   7.98e-05      3.822      0.000         0.000     0.000
C(FIPSST)[T.10]   6.829e-06   5.12e-05      0.133      0.894     -9.35e-05     0.000
C(FIPSST)[T.12]     -0.0001   7.03e-05     -2.076      0.038        -0.000 -8.18e-06
C(FIPSST)[T.13]   2.126

In [211]:
topline_mods['RV'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               RV_PC_LN   R-squared:                       0.212
Model:                            WLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                -1.391e+15
Date:                Sun, 15 Nov 2015   Prob (F-statistic):               1.00
Time:                        23:25:34   Log-Likelihood:             1.1993e+05
No. Observations:               21816   AIC:                        -2.398e+05
Df Residuals:                   21771   BIC:                        -2.394e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0006      0.000     -1.290      0.197        -0.002     0.000
C(Year)[T.1986]     -0.0002      0.000     -1.875      0.061        -0.000  1.01e-05
C(Year)[T.1987]     -0.0003   8.24e-05     -3.126      0.002        -0.000  -9.6e-05
C(Year)[T.1988]     -0.0005      0.000     -5.006      0.000        -0.001    -0.000
C(Year)[T.1989]     -0.0004      0.000     -3.460      0.001        -0.001    -0.000
C(Year)[T.1990]     -0.0001   5.25e-05     -2.275      0.023        -0.000 -1.66e-05
C(Year)[T.1991]     -0.0002   4.25e-05     -4.081      0.000        -0.000 -9.01e-05
C(Year)[T.1992]     -0.0001   4.76e-05     -2.918      0.004        -0.000 -4.56e-05
C(Year)[T.1993]   3.642e-05   3.78e-05      0.964      0.335     -3.76e-05     0.000
C(Year)[T.1994]     -0.0002    9.1e-05     -2.636      0.008        -0.000 -6.15e-05
C(Year)[T.1995]     -0.0002   6.06e-05     -3.879      0.000        -0.000    -0.000
C(Year)[T.1996]     -0.0002    5.1e-05     -3.226      0.001        -0.000 -6.46e-05
C(Year)[T.1997]     -0.0001   6.06e-05     -2.276      0.023        -0.000 -1.91e-05
C(Year)[T.1998]  -3.682e-06   6.59e-05     -0.056      0.955        -0.000     0.000
C(Year)[T.1999]  -6.515e-05   6.46e-05     -1.008      0.313        -0.000  6.15e-05
C(Year)[T.2000]  -8.966e-06    3.5e-05     -0.256      0.798     -7.76e-05  5.96e-05
C(Year)[T.2001]   2.599e-05   1.87e-05      1.391      0.164     -1.06e-05  6.26e-05
C(Year)[T.2002]  -7.467e-05      0.000     -0.623      0.533        -0.000     0.000
C(Year)[T.2003]   5.865e-05   6.06e-05      0.968      0.333     -6.01e-05     0.000
C(Year)[T.2004]     -0.0004   8.22e-05     -4.492      0.000        -0.001    -0.000
C(Year)[T.2005]      0.0002      0.000      1.651      0.099      -3.3e-05     0.000
C(Year)[T.2006]   7.328e-05   6.49e-05      1.129      0.259     -5.39e-05     0.000
C(Year)[T.2007]   5.139e-06   4.63e-05      0.111      0.912     -8.56e-05  9.59e-05
C(Year)[T.2008]      0.0002   7.85e-05      2.770      0.006      6.36e-05     0.000
C(Year)[T.2009]     -0.0002      0.000     -2.061      0.039        -0.000 -1.18e-05
C(Year)[T.2010]   5.759e-05   6.67e-05      0.863      0.388     -7.32e-05     0.000
C(FIPSST)[T.04]      0.0005      0.000      3.067      0.002         0.000     0.001
C(FIPSST)[T.05]   6.986e-05      0.000      0.528      0.598        -0.000     0.000
C(FIPSST)[T.06]      0.0005   9.02e-05      5.595      0.000         0.000     0.001
C(FIPSST)[T.08]      0.0006   9.86e-05      5.876      0.000         0.000     0.001
C(FIPSST)[T.09]  -6.213e-05   9.82e-05     -0.633      0.527        -0.000     0.000
C(FIPSST)[T.10]     -0.0001   6.47e-05     -2.265      0.024        -0.000 -1.97e-05
C(FIPSST)[T.12]      0.0004      9e-05      4.087      0.000         0.000     0.001
C(FIPSST)[T.13]      0.

In [212]:
topline_mods['ISSUER'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:            TOT_DEBT_PC   R-squared:                       0.234
Model:                            WLS   Adj. R-squared:                  0.233
Method:                 Least Squares   F-statistic:                -2.511e+15
Date:                Sun, 15 Nov 2015   Prob (F-statistic):               1.00
Time:                        23:25:35   Log-Likelihood:             1.1220e+05
No. Observations:               21816   AIC:                        -2.243e+05
Df Residuals:                   21775   BIC:                        -2.240e+05
Df Model:                          40                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0011      0.001     -1.981      0.048        -0.002 -1.13e-05
C(Year)[T.1986]     -0.0002      0.000     -1.589      0.112        -0.001  5.62e-05
C(Year)[T.1987]     -0.0004   9.07e-05     -4.665      0.000        -0.001    -0.000
C(Year)[T.1988]     -0.0008      0.000     -6.516      0.000        -0.001    -0.001
C(Year)[T.1989]     -0.0007      0.000     -5.658      0.000        -0.001    -0.000
C(Year)[T.1990]     -0.0003   4.87e-05     -5.340      0.000        -0.000    -0.000
C(Year)[T.1991]     -0.0001      0.000     -1.420      0.155        -0.000  5.63e-05
C(Year)[T.1992]  -3.636e-05   9.07e-05     -0.401      0.689        -0.000     0.000
C(Year)[T.1993]      0.0001   6.33e-05      2.072      0.038      7.07e-06     0.000
C(Year)[T.1994]     -0.0002      0.000     -1.697      0.090        -0.000  2.87e-05
C(Year)[T.1995]     -0.0003   8.06e-05     -3.385      0.001        -0.000    -0.000
C(Year)[T.1996]     -0.0002      0.000     -2.046      0.041        -0.000 -8.88e-06
C(Year)[T.1997]     -0.0001      0.000     -1.166      0.244        -0.000  9.47e-05
C(Year)[T.1998]   2.563e-06      0.000      0.024      0.981        -0.000     0.000
C(Year)[T.1999]     -0.0001   8.46e-05     -1.434      0.151        -0.000  4.45e-05
C(Year)[T.2000]     -0.0002   7.11e-05     -2.174      0.030        -0.000 -1.52e-05
C(Year)[T.2001]   5.413e-05   3.16e-05      1.711      0.087     -7.87e-06     0.000
C(Year)[T.2002]  -5.108e-06      0.000     -0.047      0.963        -0.000     0.000
C(Year)[T.2003]    2.61e-05   8.69e-05      0.300      0.764        -0.000     0.000
C(Year)[T.2004]     -0.0007      0.000     -6.873      0.000        -0.001    -0.001
C(Year)[T.2005]      0.0001   7.38e-05      1.575      0.115     -2.84e-05     0.000
C(Year)[T.2006]  -1.699e-05   7.13e-05     -0.238      0.812        -0.000     0.000
C(Year)[T.2007]  -5.261e-05   2.54e-05     -2.074      0.038        -0.000  -2.9e-06
C(Year)[T.2008]   3.594e-05   7.16e-05      0.502      0.616        -0.000     0.000
C(Year)[T.2009]     -0.0002      0.000     -1.243      0.214        -0.000   8.7e-05
C(Year)[T.2010]  -6.006e-05   6.56e-05     -0.916      0.360        -0.000  6.85e-05
C(FIPSST)[T.04]      0.0004      0.000      1.971      0.049      2.47e-06     0.001
C(FIPSST)[T.05]      0.0002      0.000      1.150      0.250        -0.000     0.000
C(FIPSST)[T.06]      0.0004      0.000      2.199      0.028      4.64e-05     0.001
C(FIPSST)[T.08]      0.0007      0.000      6.423      0.000         0.000     0.001
C(FIPSST)[T.09]      0.0002      0.000      1.370      0.171        -0.000     0.001
C(FIPSST)[T.10]     -0.0001      0.000     -0.840      0.401        -0.000     0.000
C(FIPSST)[T.12]      0.0002      0.000      1.286      0.198     -9.63e-05     0.000
C(FIPSST)[T.13]      0.

In [213]:
topline_mods['PURPOSE'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:            TOT_DEBT_PC   R-squared:                       0.247
Model:                            WLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                 3.097e+14
Date:                Sun, 15 Nov 2015   Prob (F-statistic):           1.43e-50
Time:                        23:25:35   Log-Likelihood:             1.1237e+05
No. Observations:               21816   AIC:                        -2.247e+05
Df Residuals:                   21772   BIC:                        -2.243e+05
Df Model:                          43                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0012      0.000     -3.256      0.001        -0.002    -0.000
C(Year)[T.1986]     -0.0002      0.000     -1.070      0.285        -0.000     0.000
C(Year)[T.1987]     -0.0003   4.59e-05     -7.103      0.000        -0.000    -0.000
C(Year)[T.1988]     -0.0007      0.000     -7.118      0.000        -0.001    -0.001
C(Year)[T.1989]     -0.0006   7.04e-05     -8.492      0.000        -0.001    -0.000
C(Year)[T.1990]     -0.0002   8.63e-05     -1.769      0.077        -0.000  1.65e-05
C(Year)[T.1991]  -2.195e-05   8.56e-05     -0.256      0.798        -0.000     0.000
C(Year)[T.1992]   3.895e-05   9.42e-05      0.413      0.679        -0.000     0.000
C(Year)[T.1993]      0.0002   5.49e-05      4.232      0.000         0.000     0.000
C(Year)[T.1994]     -0.0002      0.000     -1.369      0.171        -0.000  7.22e-05
C(Year)[T.1995]     -0.0002   8.46e-05     -2.716      0.007        -0.000 -6.39e-05
C(Year)[T.1996]  -8.326e-05   9.04e-05     -0.921      0.357        -0.000   9.4e-05
C(Year)[T.1997]  -8.226e-05      0.000     -0.681      0.496        -0.000     0.000
C(Year)[T.1998]   7.632e-05      0.000      0.751      0.453        -0.000     0.000
C(Year)[T.1999]  -2.033e-05    7.8e-05     -0.261      0.794        -0.000     0.000
C(Year)[T.2000]  -6.411e-06   7.24e-05     -0.089      0.929        -0.000     0.000
C(Year)[T.2001]      0.0001   2.07e-05      5.321      0.000      6.96e-05     0.000
C(Year)[T.2002]    9.91e-05      0.000      0.986      0.324     -9.79e-05     0.000
C(Year)[T.2003]      0.0003   6.71e-05      3.834      0.000         0.000     0.000
C(Year)[T.2004]     -0.0005   8.87e-05     -6.139      0.000        -0.001    -0.000
C(Year)[T.2005]      0.0004      0.000      3.037      0.002         0.000     0.001
C(Year)[T.2006]      0.0002   8.03e-05      2.873      0.004      7.33e-05     0.000
C(Year)[T.2007]      0.0002   5.88e-05      3.821      0.000         0.000     0.000
C(Year)[T.2008]      0.0002      0.000      1.790      0.073     -1.78e-05     0.000
C(Year)[T.2009]     -0.0002      0.000     -1.645      0.100        -0.000  3.91e-05
C(Year)[T.2010]      0.0001   7.63e-05      1.774      0.076     -1.42e-05     0.000
C(FIPSST)[T.04]      0.0003      0.000      1.426      0.154        -0.000     0.001
C(FIPSST)[T.05]  -6.683e-05   7.15e-05     -0.935      0.350        -0.000  7.32e-05
C(FIPSST)[T.06]      0.0004      0.000      1.932      0.053     -5.37e-06     0.001
C(FIPSST)[T.08]      0.0003      0.000      2.395      0.017      5.89e-05     0.001
C(FIPSST)[T.09]      0.0003      0.000      2.130      0.033      2.65e-05     0.001
C(FIPSST)[T.10]     -0.0002   7.47e-05     -3.199      0.001        -0.000 -9.25e-05
C(FIPSST)[T.12]   5.235e-05      0.000      0.461      0.645        -0.000     0.000
C(FIPSST)[T.13]      0.

In [214]:
topline_mods['RD_GO'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               GO_RD_LN   R-squared:                       0.092
Model:                            WLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                 5.299e+14
Date:                Sun, 15 Nov 2015   Prob (F-statistic):           2.19e-51
Time:                        23:25:35   Log-Likelihood:             1.3533e+05
No. Observations:               21816   AIC:                        -2.706e+05
Df Residuals:                   21771   BIC:                        -2.702e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0004      0.000     -2.168      0.030        -0.001 -3.49e-05
C(Year)[T.1986]   3.276e-05   3.55e-05      0.922      0.357     -3.69e-05     0.000
C(Year)[T.1987]  -4.886e-05    2.1e-05     -2.326      0.020        -9e-05 -7.69e-06
C(Year)[T.1988]     -0.0002   2.49e-05     -8.376      0.000        -0.000    -0.000
C(Year)[T.1989]     -0.0002   2.29e-05     -7.944      0.000        -0.000    -0.000
C(Year)[T.1990]  -7.867e-06   2.01e-05     -0.391      0.696     -4.73e-05  3.15e-05
C(Year)[T.1991]   1.381e-05   2.82e-05      0.490      0.624     -4.14e-05  6.91e-05
C(Year)[T.1992]   1.249e-05   2.83e-05      0.442      0.659     -4.29e-05  6.79e-05
C(Year)[T.1993]   6.025e-05   2.15e-05      2.807      0.005      1.82e-05     0.000
C(Year)[T.1994]  -4.851e-05   3.36e-05     -1.442      0.149        -0.000  1.74e-05
C(Year)[T.1995]  -5.215e-05   2.35e-05     -2.221      0.026     -9.82e-05 -6.13e-06
C(Year)[T.1996]  -3.423e-05   2.42e-05     -1.416      0.157     -8.16e-05  1.32e-05
C(Year)[T.1997]  -4.882e-05   2.91e-05     -1.676      0.094        -0.000  8.29e-06
C(Year)[T.1998]  -3.197e-05   2.36e-05     -1.357      0.175     -7.82e-05  1.42e-05
C(Year)[T.1999]  -5.131e-05    2.2e-05     -2.330      0.020     -9.45e-05 -8.15e-06
C(Year)[T.2000]  -4.543e-05   1.98e-05     -2.290      0.022     -8.43e-05 -6.55e-06
C(Year)[T.2001]   1.215e-05   8.19e-06      1.483      0.138     -3.91e-06  2.82e-05
C(Year)[T.2002]  -4.364e-05   2.45e-05     -1.782      0.075     -9.16e-05  4.36e-06
C(Year)[T.2003]  -2.234e-05   2.76e-05     -0.810      0.418     -7.64e-05  3.17e-05
C(Year)[T.2004]     -0.0002   2.33e-05     -6.523      0.000        -0.000    -0.000
C(Year)[T.2005]    1.25e-05   2.36e-05      0.529      0.597     -3.38e-05  5.88e-05
C(Year)[T.2006]   -3.46e-06    2.2e-05     -0.157      0.875     -4.67e-05  3.98e-05
C(Year)[T.2007]   1.653e-05   4.11e-05      0.402      0.687      -6.4e-05   9.7e-05
C(Year)[T.2008]   2.465e-05   2.84e-05      0.868      0.385      -3.1e-05  8.03e-05
C(Year)[T.2009]  -6.301e-05    3.9e-05     -1.615      0.106        -0.000  1.35e-05
C(Year)[T.2010]  -3.644e-05   2.21e-05     -1.646      0.100     -7.98e-05  6.96e-06
C(FIPSST)[T.04]   -5.98e-05    4.9e-05     -1.220      0.222        -0.000  3.63e-05
C(FIPSST)[T.05]   5.518e-05   4.96e-05      1.112      0.266     -4.21e-05     0.000
C(FIPSST)[T.06]  -7.281e-05   3.54e-05     -2.055      0.040        -0.000 -3.35e-06
C(FIPSST)[T.08]  -4.634e-05    3.9e-05     -1.190      0.234        -0.000     3e-05
C(FIPSST)[T.09]   8.265e-05   4.41e-05      1.874      0.061     -3.81e-06     0.000
C(FIPSST)[T.10]  -1.827e-06   2.29e-05     -0.080      0.936     -4.67e-05   4.3e-05
C(FIPSST)[T.12]  -5.866e-05   4.35e-05     -1.349      0.177        -0.000  2.65e-05
C(FIPSST)[T.13]   1.684

In [215]:
topline_mods['RD_GO'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               GO_RD_LN   R-squared:                       0.092
Model:                            WLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                 5.299e+14
Date:                Sun, 15 Nov 2015   Prob (F-statistic):           2.19e-51
Time:                        23:25:35   Log-Likelihood:             1.3533e+05
No. Observations:               21816   AIC:                        -2.706e+05
Df Residuals:                   21771   BIC:                        -2.702e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0004      0.000     -2.168      0.030        -0.001 -3.49e-05
C(Year)[T.1986]   3.276e-05   3.55e-05      0.922      0.357     -3.69e-05     0.000
C(Year)[T.1987]  -4.886e-05    2.1e-05     -2.326      0.020        -9e-05 -7.69e-06
C(Year)[T.1988]     -0.0002   2.49e-05     -8.376      0.000        -0.000    -0.000
C(Year)[T.1989]     -0.0002   2.29e-05     -7.944      0.000        -0.000    -0.000
C(Year)[T.1990]  -7.867e-06   2.01e-05     -0.391      0.696     -4.73e-05  3.15e-05
C(Year)[T.1991]   1.381e-05   2.82e-05      0.490      0.624     -4.14e-05  6.91e-05
C(Year)[T.1992]   1.249e-05   2.83e-05      0.442      0.659     -4.29e-05  6.79e-05
C(Year)[T.1993]   6.025e-05   2.15e-05      2.807      0.005      1.82e-05     0.000
C(Year)[T.1994]  -4.851e-05   3.36e-05     -1.442      0.149        -0.000  1.74e-05
C(Year)[T.1995]  -5.215e-05   2.35e-05     -2.221      0.026     -9.82e-05 -6.13e-06
C(Year)[T.1996]  -3.423e-05   2.42e-05     -1.416      0.157     -8.16e-05  1.32e-05
C(Year)[T.1997]  -4.882e-05   2.91e-05     -1.676      0.094        -0.000  8.29e-06
C(Year)[T.1998]  -3.197e-05   2.36e-05     -1.357      0.175     -7.82e-05  1.42e-05
C(Year)[T.1999]  -5.131e-05    2.2e-05     -2.330      0.020     -9.45e-05 -8.15e-06
C(Year)[T.2000]  -4.543e-05   1.98e-05     -2.290      0.022     -8.43e-05 -6.55e-06
C(Year)[T.2001]   1.215e-05   8.19e-06      1.483      0.138     -3.91e-06  2.82e-05
C(Year)[T.2002]  -4.364e-05   2.45e-05     -1.782      0.075     -9.16e-05  4.36e-06
C(Year)[T.2003]  -2.234e-05   2.76e-05     -0.810      0.418     -7.64e-05  3.17e-05
C(Year)[T.2004]     -0.0002   2.33e-05     -6.523      0.000        -0.000    -0.000
C(Year)[T.2005]    1.25e-05   2.36e-05      0.529      0.597     -3.38e-05  5.88e-05
C(Year)[T.2006]   -3.46e-06    2.2e-05     -0.157      0.875     -4.67e-05  3.98e-05
C(Year)[T.2007]   1.653e-05   4.11e-05      0.402      0.687      -6.4e-05   9.7e-05
C(Year)[T.2008]   2.465e-05   2.84e-05      0.868      0.385      -3.1e-05  8.03e-05
C(Year)[T.2009]  -6.301e-05    3.9e-05     -1.615      0.106        -0.000  1.35e-05
C(Year)[T.2010]  -3.644e-05   2.21e-05     -1.646      0.100     -7.98e-05  6.96e-06
C(FIPSST)[T.04]   -5.98e-05    4.9e-05     -1.220      0.222        -0.000  3.63e-05
C(FIPSST)[T.05]   5.518e-05   4.96e-05      1.112      0.266     -4.21e-05     0.000
C(FIPSST)[T.06]  -7.281e-05   3.54e-05     -2.055      0.040        -0.000 -3.35e-06
C(FIPSST)[T.08]  -4.634e-05    3.9e-05     -1.190      0.234        -0.000     3e-05
C(FIPSST)[T.09]   8.265e-05   4.41e-05      1.874      0.061     -3.81e-06     0.000
C(FIPSST)[T.10]  -1.827e-06   2.29e-05     -0.080      0.936     -4.67e-05   4.3e-05
C(FIPSST)[T.12]  -5.866e-05   4.35e-05     -1.349      0.177        -0.000  2.65e-05
C(FIPSST)[T.13]   1.684

Let's try it on just the subset of folks with positive total debt.

In [216]:
#Subset to positive total debt
data_pos_debt=data_d[data_d['TOT_DEBT']>0]

# topline_mods_debt_sub={'GO':smf.ols(formula=topline_specs['GO'],data=data_pos_debt).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']}),
#               'RV':smf.ols(formula=topline_specs['RV'],data=data_pos_debt).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']}),
#               'ISSUER':smf.ols(formula=topline_specs['ISSUER'],data=data_pos_debt).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']}),
#               'PURPOSE':smf.ols(formula=topline_specs['PURPOSE'],data=data_pos_debt).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']}),
#               'RD_GO':smf.ols(formula=topline_specs['RD_GO'],data=data_pos_debt).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']}),
#               'RD_RV':smf.ols(formula=topline_specs['RD_RV'],data=data_pos_debt).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']})}

topline_mods_debt_sub={'GO':smf.wls(formula=topline_specs['GO'],data=data_pos_debt,weights=data_pos_debt['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']}),
              'RV':smf.wls(formula=topline_specs['RV'],data=data_pos_debt,weights=data_pos_debt['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']}),
              'ISSUER':smf.wls(formula=topline_specs['ISSUER'],data=data_pos_debt,weights=data_pos_debt['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']}),
              'PURPOSE':smf.wls(formula=topline_specs['PURPOSE'],data=data_pos_debt,weights=data_pos_debt['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']}),
              'RD_GO':smf.wls(formula=topline_specs['RD_GO'],data=data_pos_debt,weights=data_pos_debt['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']}),
              'RD_RV':smf.wls(formula=topline_specs['RD_RV'],data=data_pos_debt,weights=data_pos_debt['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_pos_debt['BEA_INT']})}

topline_mods_debt_sub['GO'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               GO_PC_LN   R-squared:                       0.146
Model:                            WLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                -5.380e+15
Date:                Sun, 15 Nov 2015   Prob (F-statistic):               1.00
Time:                        23:25:37   Log-Likelihood:             1.0820e+05
No. Observations:               19390   AIC:                        -2.163e+05
Df Residuals:                   19348   BIC:                        -2.160e+05
Df Model:                          41                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0007      0.000     -1.930      0.054        -0.001  1.01e-05
C(Year)[T.1986]  -1.706e-05   5.17e-05     -0.330      0.742        -0.000  8.43e-05
C(Year)[T.1987]     -0.0002   2.39e-05     -8.457      0.000        -0.000    -0.000
C(Year)[T.1990]     -0.0002   2.55e-05     -7.123      0.000        -0.000    -0.000
C(Year)[T.1991]  -2.663e-05   5.49e-05     -0.485      0.627        -0.000  8.09e-05
C(Year)[T.1992]   5.279e-05   6.18e-05      0.854      0.393     -6.84e-05     0.000
C(Year)[T.1993]   7.339e-05   6.49e-05      1.130      0.258     -5.39e-05     0.000
C(Year)[T.1994]  -2.431e-05   7.56e-05     -0.322      0.748        -0.000     0.000
C(Year)[T.1995]     -0.0001   4.45e-05     -2.390      0.017        -0.000 -1.91e-05
C(Year)[T.1996]  -6.308e-05   4.33e-05     -1.456      0.145        -0.000  2.18e-05
C(Year)[T.1997]  -1.894e-05   6.88e-05     -0.276      0.783        -0.000     0.000
C(Year)[T.1998]   4.779e-06   4.28e-05      0.112      0.911      -7.9e-05  8.86e-05
C(Year)[T.1999]  -7.524e-05   4.11e-05     -1.830      0.067        -0.000  5.33e-06
C(Year)[T.2000]     -0.0002   7.12e-05     -2.204      0.028        -0.000 -1.74e-05
C(Year)[T.2001]   4.914e-05   2.12e-05      2.316      0.021      7.55e-06  9.07e-05
C(Year)[T.2002]   4.429e-05   5.09e-05      0.870      0.384     -5.55e-05     0.000
C(Year)[T.2003]  -7.908e-06   5.69e-05     -0.139      0.889        -0.000     0.000
C(Year)[T.2005]  -6.962e-05    8.3e-05     -0.839      0.401        -0.000   9.3e-05
C(Year)[T.2006]     -0.0001   6.42e-05     -1.701      0.089        -0.000  1.66e-05
C(Year)[T.2007]  -7.681e-05   8.28e-05     -0.928      0.354        -0.000  8.54e-05
C(Year)[T.2008]     -0.0001   6.95e-05     -1.992      0.046        -0.000 -2.25e-06
C(Year)[T.2009]   1.217e-05   9.02e-05      0.135      0.893        -0.000     0.000
C(Year)[T.2010]     -0.0001    5.5e-05     -1.931      0.054        -0.000  1.62e-06
C(FIPSST)[T.04]     -0.0001      0.000     -1.228      0.219        -0.000  7.48e-05
C(FIPSST)[T.05]   4.757e-05   4.74e-05      1.004      0.315     -4.53e-05     0.000
C(FIPSST)[T.06]  -2.464e-05   6.69e-05     -0.368      0.713        -0.000     0.000
C(FIPSST)[T.08]   1.686e-05   5.19e-05      0.325      0.745     -8.49e-05     0.000
C(FIPSST)[T.09]      0.0004   5.28e-05      6.632      0.000         0.000     0.000
C(FIPSST)[T.10]  -5.325e-05   4.89e-05     -1.089      0.276        -0.000  4.26e-05
C(FIPSST)[T.12]     -0.0002   7.83e-05     -2.234      0.026        -0.000 -2.14e-05
C(FIPSST)[T.13]   1.911e-05   5.04e-05      0.379      0.705     -7.97e-05     0.000
C(FIPSST)[T.15]      0.0001   7.58e-05      1.851      0.064     -8.28e-06     0.000
C(FIPSST)[T.16]     -0.0002   6.39e-05     -2.622      0.009        -0.000 -4.23e-05
C(FIPSST)[T.17]   2.207

In [217]:
topline_mods_debt_sub['RV'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               RV_PC_LN   R-squared:                       0.210
Model:                            WLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                 1.713e+17
Date:                Sun, 15 Nov 2015   Prob (F-statistic):           3.62e-60
Time:                        23:25:38   Log-Likelihood:             1.0570e+05
No. Observations:               19390   AIC:                        -2.113e+05
Df Residuals:                   19348   BIC:                        -2.110e+05
Df Model:                          41                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0003      0.000     -0.895      0.371        -0.001     0.000
C(Year)[T.1986]     -0.0003      0.000     -1.945      0.052        -0.001  1.97e-06
C(Year)[T.1987]     -0.0003    8.9e-05     -3.216      0.001        -0.000    -0.000
C(Year)[T.1990]     -0.0002   6.32e-05     -3.190      0.001        -0.000 -7.78e-05
C(Year)[T.1991]     -0.0002   4.75e-05     -4.748      0.000        -0.000    -0.000
C(Year)[T.1992]     -0.0002   4.75e-05     -4.277      0.000        -0.000    -0.000
C(Year)[T.1993]  -3.644e-05   4.58e-05     -0.795      0.427        -0.000  5.34e-05
C(Year)[T.1994]     -0.0003   9.11e-05     -3.436      0.001        -0.000    -0.000
C(Year)[T.1995]     -0.0003   7.77e-05     -4.071      0.000        -0.000    -0.000
C(Year)[T.1996]     -0.0002   6.15e-05     -3.920      0.000        -0.000    -0.000
C(Year)[T.1997]     -0.0002   6.36e-05     -3.210      0.001        -0.000 -7.95e-05
C(Year)[T.1998]  -8.068e-05   6.99e-05     -1.154      0.248        -0.000  5.63e-05
C(Year)[T.1999]     -0.0002   6.06e-05     -2.487      0.013        -0.000 -3.19e-05
C(Year)[T.2000]     -0.0001   3.63e-05     -2.761      0.006        -0.000 -2.91e-05
C(Year)[T.2001]   7.925e-06   1.77e-05      0.448      0.654     -2.68e-05  4.26e-05
C(Year)[T.2002]     -0.0001   9.98e-05     -1.411      0.158        -0.000  5.48e-05
C(Year)[T.2003]  -4.055e-05   4.56e-05     -0.889      0.374        -0.000  4.88e-05
C(Year)[T.2005]   6.746e-05   9.94e-05      0.679      0.497        -0.000     0.000
C(Year)[T.2006]  -1.954e-05   5.81e-05     -0.336      0.737        -0.000  9.43e-05
C(Year)[T.2007]  -9.294e-05   5.01e-05     -1.854      0.064        -0.000  5.31e-06
C(Year)[T.2008]      0.0002   8.11e-05      2.534      0.011      4.66e-05     0.000
C(Year)[T.2009]     -0.0003      0.000     -2.611      0.009        -0.001  -7.2e-05
C(Year)[T.2010]  -3.595e-05   4.53e-05     -0.793      0.428        -0.000  5.29e-05
C(FIPSST)[T.04]      0.0003      0.000      1.627      0.104     -6.86e-05     0.001
C(FIPSST)[T.05]  -2.359e-05   4.21e-05     -0.560      0.576        -0.000   5.9e-05
C(FIPSST)[T.06]      0.0004      0.000      2.461      0.014      8.82e-05     0.001
C(FIPSST)[T.08]      0.0005   8.74e-05      5.901      0.000         0.000     0.001
C(FIPSST)[T.09]     -0.0002      0.000     -0.965      0.334        -0.001     0.000
C(FIPSST)[T.10]     -0.0003      0.000     -2.317      0.020        -0.000 -4.04e-05
C(FIPSST)[T.12]      0.0003      0.000      2.426      0.015      5.01e-05     0.000
C(FIPSST)[T.13]      0.0002   3.89e-05      5.041      0.000         0.000     0.000
C(FIPSST)[T.15]     -0.0004      0.000     -3.464      0.001        -0.001    -0.000
C(FIPSST)[T.16]     -0.0002      0.000     -1.293      0.196        -0.000  8.21e-05
C(FIPSST)[T.17]  -3.436

In [218]:
topline_mods_debt_sub['ISSUER'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:            TOT_DEBT_PC   R-squared:                       0.199
Model:                            WLS   Adj. R-squared:                  0.197
Method:                 Least Squares   F-statistic:                 1.315e+15
Date:                Sun, 15 Nov 2015   Prob (F-statistic):           9.05e-53
Time:                        23:25:38   Log-Likelihood:                 98742.
No. Observations:               19390   AIC:                        -1.974e+05
Df Residuals:                   19347   BIC:                        -1.971e+05
Df Model:                          42                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0012      0.001     -1.768      0.077        -0.002     0.000
C(Year)[T.1986]     -0.0002      0.000     -1.422      0.155        -0.001  8.74e-05
C(Year)[T.1987]     -0.0005      0.000     -4.553      0.000        -0.001    -0.000
C(Year)[T.1990]     -0.0003   6.85e-05     -4.693      0.000        -0.000    -0.000
C(Year)[T.1991]     -0.0002      0.000     -2.279      0.023        -0.000 -3.24e-05
C(Year)[T.1992]  -8.251e-05      0.000     -0.745      0.456        -0.000     0.000
C(Year)[T.1993]     6.1e-05   6.14e-05      0.993      0.320     -5.93e-05     0.000
C(Year)[T.1994]     -0.0002      0.000     -1.773      0.076        -0.001  2.64e-05
C(Year)[T.1995]     -0.0003    9.1e-05     -3.709      0.000        -0.001    -0.000
C(Year)[T.1996]     -0.0003      0.000     -2.441      0.015        -0.000 -5.24e-05
C(Year)[T.1997]     -0.0002      0.000     -1.393      0.163        -0.000  7.37e-05
C(Year)[T.1998]  -1.241e-05      0.000     -0.101      0.920        -0.000     0.000
C(Year)[T.1999]     -0.0002   8.76e-05     -2.146      0.032        -0.000 -1.63e-05
C(Year)[T.2000]     -0.0002   9.64e-05     -2.470      0.014        -0.000 -4.92e-05
C(Year)[T.2001]   6.004e-05   4.22e-05      1.423      0.155     -2.26e-05     0.000
C(Year)[T.2002]  -5.861e-05      0.000     -0.369      0.712        -0.000     0.000
C(Year)[T.2003]  -3.774e-05      0.000     -0.375      0.708        -0.000     0.000
C(Year)[T.2005]   7.033e-06   9.49e-05      0.074      0.941        -0.000     0.000
C(Year)[T.2006]     -0.0001   9.83e-05     -1.434      0.151        -0.000  5.17e-05
C(Year)[T.2007]     -0.0002   6.15e-05     -2.770      0.006        -0.000 -4.99e-05
C(Year)[T.2008]  -1.576e-05      0.000     -0.118      0.906        -0.000     0.000
C(Year)[T.2009]     -0.0002      0.000     -0.843      0.399        -0.001     0.000
C(Year)[T.2010]     -0.0001   9.84e-05     -1.335      0.182        -0.000  6.15e-05
C(FIPSST)[T.04]      0.0004      0.000      2.662      0.008         0.000     0.001
C(FIPSST)[T.05]      0.0003      0.000      1.528      0.127     -8.43e-05     0.001
C(FIPSST)[T.06]      0.0005      0.000      3.474      0.001         0.000     0.001
C(FIPSST)[T.08]      0.0008   8.52e-05      9.806      0.000         0.001     0.001
C(FIPSST)[T.09]      0.0003      0.000      2.206      0.027      3.66e-05     0.001
C(FIPSST)[T.10]  -3.281e-05      0.000     -0.315      0.752        -0.000     0.000
C(FIPSST)[T.12]      0.0002      0.000      1.774      0.076     -2.59e-05     0.001
C(FIPSST)[T.13]      0.0003   5.44e-05      4.643      0.000         0.000     0.000
C(FIPSST)[T.15]   9.931e-06      0.000      0.046      0.964        -0.000     0.000
C(FIPSST)[T.16]  -6.069e-05      0.000     -0.353      0.724        -0.000     0.000
C(FIPSST)[T.17]      0.

In [219]:
topline_mods_debt_sub['PURPOSE'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:            TOT_DEBT_PC   R-squared:                       0.209
Model:                            WLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                 7.437e+14
Date:                Sun, 15 Nov 2015   Prob (F-statistic):           6.67e-52
Time:                        23:25:38   Log-Likelihood:                 98861.
No. Observations:               19390   AIC:                        -1.976e+05
Df Residuals:                   19348   BIC:                        -1.973e+05
Df Model:                          41                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0011      0.001     -1.746      0.081        -0.002     0.000
C(Year)[T.1986]     -0.0002      0.000     -1.351      0.177        -0.001  9.23e-05
C(Year)[T.1987]     -0.0004   6.03e-05     -7.290      0.000        -0.001    -0.000
C(Year)[T.1990]     -0.0004    6.6e-05     -6.283      0.000        -0.001    -0.000
C(Year)[T.1991]     -0.0002   5.98e-05     -3.581      0.000        -0.000 -9.69e-05
C(Year)[T.1992]     -0.0001   7.95e-05     -1.457      0.145        -0.000     4e-05
C(Year)[T.1993]   4.697e-05   6.22e-05      0.755      0.450     -7.49e-05     0.000
C(Year)[T.1994]     -0.0003      0.000     -2.854      0.004        -0.001    -0.000
C(Year)[T.1995]     -0.0004   6.36e-05     -6.085      0.000        -0.001    -0.000
C(Year)[T.1996]     -0.0003   7.35e-05     -3.738      0.000        -0.000    -0.000
C(Year)[T.1997]     -0.0003      0.000     -2.343      0.019        -0.000 -4.23e-05
C(Year)[T.1998]  -9.264e-05   8.27e-05     -1.120      0.263        -0.000  6.95e-05
C(Year)[T.1999]     -0.0002   6.24e-05     -3.325      0.001        -0.000 -8.51e-05
C(Year)[T.2000]     -0.0002   9.81e-05     -2.427      0.015        -0.000 -4.58e-05
C(Year)[T.2001]   7.375e-05   2.65e-05      2.781      0.005      2.18e-05     0.000
C(Year)[T.2002]  -9.873e-05      0.000     -0.872      0.383        -0.000     0.000
C(Year)[T.2003]  -1.545e-06   9.08e-05     -0.017      0.986        -0.000     0.000
C(Year)[T.2005]   6.222e-05      0.000      0.382      0.703        -0.000     0.000
C(Year)[T.2006]     -0.0001      0.000     -0.914      0.361        -0.000     0.000
C(Year)[T.2007]     -0.0001      0.000     -1.072      0.284        -0.000  9.06e-05
C(Year)[T.2008]  -4.269e-06      0.000     -0.029      0.977        -0.000     0.000
C(Year)[T.2009]     -0.0002      0.000     -1.191      0.234        -0.001     0.000
C(Year)[T.2010]     -0.0001      0.000     -1.027      0.305        -0.000     0.000
C(FIPSST)[T.04]      0.0003      0.000      1.401      0.161        -0.000     0.001
C(FIPSST)[T.05]   9.559e-05      0.000      0.562      0.574        -0.000     0.000
C(FIPSST)[T.06]      0.0004      0.000      3.466      0.001         0.000     0.001
C(FIPSST)[T.08]      0.0004      0.000      3.418      0.001         0.000     0.001
C(FIPSST)[T.09]      0.0004      0.000      2.803      0.005         0.000     0.001
C(FIPSST)[T.10]  -6.118e-05      0.000     -0.415      0.678        -0.000     0.000
C(FIPSST)[T.12]   6.105e-05      0.000      0.496      0.620        -0.000     0.000
C(FIPSST)[T.13]      0.0003      0.000      2.403      0.016      4.67e-05     0.000
C(FIPSST)[T.15]  -3.888e-05      0.000     -0.295      0.768        -0.000     0.000
C(FIPSST)[T.16]     -0.0002      0.000     -0.952      0.341        -0.001     0.000
C(FIPSST)[T.17]   5.855

In [220]:
topline_mods_debt_sub['RD_GO'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               GO_RD_LN   R-squared:                       0.078
Model:                            WLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                 6.360e+14
Date:                Sun, 15 Nov 2015   Prob (F-statistic):           1.16e-51
Time:                        23:25:38   Log-Likelihood:             1.1922e+05
No. Observations:               19390   AIC:                        -2.384e+05
Df Residuals:                   19348   BIC:                        -2.380e+05
Df Model:                          41                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0003      0.000     -2.026      0.043        -0.001 -9.52e-06
C(Year)[T.1986]   2.517e-05   3.55e-05      0.708      0.479     -4.45e-05  9.48e-05
C(Year)[T.1987]  -5.495e-05   2.15e-05     -2.560      0.010      -9.7e-05 -1.29e-05
C(Year)[T.1990]  -2.926e-05   2.26e-05     -1.293      0.196     -7.36e-05  1.51e-05
C(Year)[T.1991]   2.993e-06   3.04e-05      0.098      0.922     -5.66e-05  6.26e-05
C(Year)[T.1992]   4.195e-06   3.12e-05      0.135      0.893     -5.69e-05  6.53e-05
C(Year)[T.1993]   4.505e-05   2.24e-05      2.007      0.045      1.05e-06   8.9e-05
C(Year)[T.1994]  -5.742e-05    3.9e-05     -1.472      0.141        -0.000   1.9e-05
C(Year)[T.1995]  -6.721e-05   2.39e-05     -2.810      0.005        -0.000 -2.03e-05
C(Year)[T.1996]   -4.65e-05   2.41e-05     -1.926      0.054     -9.38e-05  8.14e-07
C(Year)[T.1997]  -5.668e-05    2.9e-05     -1.955      0.051        -0.000  1.56e-07
C(Year)[T.1998]  -4.334e-05   2.32e-05     -1.870      0.061     -8.88e-05  2.08e-06
C(Year)[T.1999]  -6.513e-05   2.27e-05     -2.869      0.004        -0.000 -2.06e-05
C(Year)[T.2000]  -6.374e-05   2.09e-05     -3.044      0.002        -0.000 -2.27e-05
C(Year)[T.2001]   8.436e-06   9.25e-06      0.912      0.362      -9.7e-06  2.66e-05
C(Year)[T.2002]  -5.112e-05   2.83e-05     -1.804      0.071        -0.000  4.43e-06
C(Year)[T.2003]  -4.005e-05   2.38e-05     -1.685      0.092     -8.66e-05  6.54e-06
C(Year)[T.2005]  -1.348e-05   2.46e-05     -0.549      0.583     -6.16e-05  3.47e-05
C(Year)[T.2006]  -2.597e-05   2.38e-05     -1.090      0.276     -7.26e-05  2.07e-05
C(Year)[T.2007]  -5.404e-06   3.69e-05     -0.147      0.883     -7.76e-05  6.68e-05
C(Year)[T.2008]   1.949e-05   3.49e-05      0.558      0.577      -4.9e-05   8.8e-05
C(Year)[T.2009]  -6.817e-05   5.24e-05     -1.301      0.193        -0.000  3.45e-05
C(Year)[T.2010]  -5.414e-05   2.28e-05     -2.371      0.018     -9.89e-05 -9.38e-06
C(FIPSST)[T.04]  -9.988e-05   5.53e-05     -1.807      0.071        -0.000  8.43e-06
C(FIPSST)[T.05]   4.718e-05   3.39e-05      1.393      0.164     -1.92e-05     0.000
C(FIPSST)[T.06]  -7.156e-05   4.24e-05     -1.686      0.092        -0.000  1.16e-05
C(FIPSST)[T.08]  -5.793e-05   3.49e-05     -1.659      0.097        -0.000  1.05e-05
C(FIPSST)[T.09]      0.0001   3.79e-05      3.250      0.001      4.89e-05     0.000
C(FIPSST)[T.10]  -2.389e-05   3.04e-05     -0.785      0.432     -8.35e-05  3.57e-05
C(FIPSST)[T.12]  -6.265e-05    4.2e-05     -1.491      0.136        -0.000  1.97e-05
C(FIPSST)[T.13]   8.828e-06   3.41e-05      0.259      0.796      -5.8e-05  7.57e-05
C(FIPSST)[T.15]      0.0003   3.64e-05      8.405      0.000         0.000     0.000
C(FIPSST)[T.16]  -9.259e-05   3.96e-05     -2.336      0.019        -0.000 -1.49e-05
C(FIPSST)[T.17]  -1.365

In [221]:
topline_mods_debt_sub['RD_RV'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               RV_RD_LN   R-squared:                       0.130
Model:                            WLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                -5.266e+14
Date:                Sun, 15 Nov 2015   Prob (F-statistic):               1.00
Time:                        23:25:38   Log-Likelihood:             1.1965e+05
No. Observations:               19390   AIC:                        -2.392e+05
Df Residuals:                   19348   BIC:                        -2.389e+05
Df Model:                          41                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept           -0.0004      0.000     -2.645      0.008        -0.001 -9.58e-05
C(Year)[T.1986]     -0.0001    7.3e-05     -1.784      0.074        -0.000  1.28e-05
C(Year)[T.1987]     -0.0001   4.84e-05     -2.625      0.009        -0.000 -3.22e-05
C(Year)[T.1990]  -4.178e-05   2.21e-05     -1.894      0.058      -8.5e-05  1.45e-06
C(Year)[T.1991]     -0.0001   2.45e-05     -4.356      0.000        -0.000 -5.87e-05
C(Year)[T.1992]     -0.0001   3.64e-05     -4.014      0.000        -0.000 -7.48e-05
C(Year)[T.1993]  -3.782e-06   1.28e-05     -0.295      0.768     -2.89e-05  2.13e-05
C(Year)[T.1994]     -0.0002    4.9e-05     -4.536      0.000        -0.000    -0.000
C(Year)[T.1995]     -0.0002   3.91e-05     -4.043      0.000        -0.000 -8.15e-05
C(Year)[T.1996]     -0.0001    3.3e-05     -3.858      0.000        -0.000 -6.26e-05
C(Year)[T.1997]     -0.0002    4.1e-05     -3.936      0.000        -0.000  -8.1e-05
C(Year)[T.1998]     -0.0001   3.92e-05     -2.790      0.005        -0.000 -3.25e-05
C(Year)[T.1999]     -0.0001   2.72e-05     -4.339      0.000        -0.000 -6.46e-05
C(Year)[T.2000]  -2.547e-05   8.04e-06     -3.168      0.002     -4.12e-05 -9.71e-06
C(Year)[T.2001]  -1.405e-06   5.06e-06     -0.277      0.782     -1.13e-05  8.52e-06
C(Year)[T.2002]     -0.0002   3.94e-05     -4.140      0.000        -0.000  -8.6e-05
C(Year)[T.2003]   -5.68e-05   1.95e-05     -2.912      0.004      -9.5e-05 -1.86e-05
C(Year)[T.2005]   6.705e-05   2.76e-05      2.430      0.015       1.3e-05     0.000
C(Year)[T.2006]   3.591e-05   1.78e-05      2.016      0.044      9.91e-07  7.08e-05
C(Year)[T.2007]  -2.444e-06   1.32e-05     -0.185      0.853     -2.83e-05  2.34e-05
C(Year)[T.2008]      0.0002   5.22e-05      4.197      0.000         0.000     0.000
C(Year)[T.2009]     -0.0003   5.44e-05     -4.727      0.000        -0.000    -0.000
C(Year)[T.2010]   -2.15e-05   2.41e-05     -0.893      0.372     -6.87e-05  2.57e-05
C(FIPSST)[T.04]   1.056e-05   6.56e-05      0.161      0.872        -0.000     0.000
C(FIPSST)[T.05]   6.141e-06   1.91e-05      0.321      0.748     -3.13e-05  4.36e-05
C(FIPSST)[T.06]   2.233e-05   3.54e-05      0.630      0.529     -4.71e-05  9.18e-05
C(FIPSST)[T.08]   1.089e-05    3.4e-05      0.320      0.749     -5.57e-05  7.75e-05
C(FIPSST)[T.09]  -9.481e-05   4.59e-05     -2.066      0.039        -0.000 -4.87e-06
C(FIPSST)[T.10]     -0.0001   3.07e-05     -4.110      0.000        -0.000  -6.6e-05
C(FIPSST)[T.12]    8.65e-05   2.15e-05      4.026      0.000      4.44e-05     0.000
C(FIPSST)[T.13]   3.823e-05   1.48e-05      2.580      0.010      9.19e-06  6.73e-05
C(FIPSST)[T.15]     -0.0001   3.24e-05     -3.676      0.000        -0.000 -5.56e-05
C(FIPSST)[T.16]     -0.0002   3.88e-05     -4.653      0.000        -0.000    -0.000
C(FIPSST)[T.17]  -5.206

In [222]:
len(data_d[pd.isnull(data_d).any(axis=1)])

0

## Estimation with a Standardized Set

It would be useful to take scale differences out of the equation by standardizing non-categorical variables.  Let's identify which variables need to standardized.

In [236]:
## Variables to standardize
std_vars=debt_vars+pc_debt_vars+prop_debt_vars+rd_debt_vars+tprop_vars['ISSUER']+tprop_vars['PURPOSE']+\
         tot_debt_vars+fiscal_vars+['DENSITY','DIVERSITY','HSLD_PERS','MANU_RES','PC_INC','PC_SSI','POP65',\
                                    'POPGROWTH','POVERTY','PRE1940','PVT_SCH','PYOUNG','RESPOP','RESPOP2',\
                                    'RETL_RES','SERV_RES','TOT_DEBT_PC','CTY_INTEREST','EMP_RES','REAL_RATE',\
                                    'R_CTY_INT','R_CTY_INT_DIFF','REAL_RATE_CHANGE','IGR_ST','TOT_DEBT_OUTST']

#Capture those left out in the cold if we are going logged dependents
remaining_vars=[var for var in data_d.columns if var not in std_vars]

#Update variables to be standardized
std_vars=std_vars+[var for var in remaining_vars if (var.startswith('GO')) | (var.startswith('RV'))]

print sorted([var for var in data_d.columns if var not in std_vars])

['ASMT_L', 'BEA', 'BEA_INT', 'BIN_REC', 'BOTH', 'FIPS', 'FIPSST', 'GP_LMT', 'INFLAT', 'LIMITS', 'RATE_L', 'SC_LMT', 'STATE', 'TOT_DEBT_PC_LN', 'TYPE1', 'TYPE2', 'TYPE2_Y', 'Year', 'defl']


To remain consistent with our clustered standard errors by region, our standardization method will subtract from each float type variable the average value for the BEA region, and that difference will be normalized by dividing it by the standard deviation for the region.  If no variation exists, the standard deviation is zero.  In this eventuality, the function (`std_val`) will return zero.  The point of standardization is to express a scale-invariant distance from central tendency.  No such distance can exist if no variation occurs.

In [224]:
# #Capture copy of data_d
# data_std=data_d.copy(deep=True)

# #Capture set of years
# yrs=sorted(set(data_std['Year']))

# #Capture set of BEA regions
# bea_regs=sorted(set(data_std['BEA']))

# #Set index
# data_std.set_index(['Year','BEA'],inplace=True)

# #Sort index
# data_std.sortlevel(0,inplace=True)

# #Define function to return a standardized value or zero
# def std_val(x):
#     x_arr=x.values
#     if np.std(x_arr)>0:
#         return (x_arr - np.mean(x_arr)) / np.std(x_arr)
#     else:
#         return 0

# #Create container for std subset
# std_subs=[]

# #For each year...
# for yr in yrs:
#     print '***',yr,'***'
#     #...for each region...
#     for br in bea_regs:
#         print '>>BEA Region:',br
#         #...capture the subset...
#         tmp_sub=data_std.ix[(yr,br)]
#         #...and for each variable...
#         for var in std_vars:
#             #...capture the series as a numpy array...
#             tmp_arr=tmp_sub[var].values
#             #...calculate the mean and std...
#             tmp_mean=tmp_arr.mean()
#             tmp_std=tmp_arr.std()
#             #...standardize the array...
#             tmp_arr_std=(tmp_arr-tmp_mean)/tmp_std
#             #...assign the standardized array back as the original variable
#             tmp_sub[var]=tmp_arr_std
#         #...and throw it in std_subs
#         std_subs.append(tmp_sub)
        
# #Concatenate back together
# data_std=pd.concat(std_subs)

# #Sort index
# data_std.sortlevel(0,inplace=True)
            
# data_std.head()

In [225]:
# #Fill missing with zero
# data_std=data_std.fillna(0).reset_index()

# #Write to disk
# data_std.to_csv('../data/debt_mod_std.csv')

In [226]:
data_std=pd.read_csv('../data/debt_mod_std.csv')

Let's rerun the models with our standardized data.  

In [227]:
# topline_mods_std={'GO':smf.ols(formula=topline_specs['GO'],data=data_std).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']}),
#               'RV':smf.ols(formula=topline_specs['RV'],data=data_std).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']}),
#               'ISSUER':smf.ols(formula=topline_specs['ISSUER'],data=data_std).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']}),
#               'PURPOSE':smf.ols(formula=topline_specs['PURPOSE'],data=data_std).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']}),
#               'RD_GO':smf.ols(formula=topline_specs['RD_GO'],data=data_std).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']}),
#               'RD_RV':smf.ols(formula=topline_specs['RD_RV'],data=data_std).\
#                       fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']})}

topline_mods_std={'GO':smf.wls(formula=topline_specs['GO'],data=data_std,weights=data_std['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']}),
              'RV':smf.wls(formula=topline_specs['RV'],data=data_std,weights=data_std['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']}),
              'ISSUER':smf.wls(formula=topline_specs['ISSUER'],data=data_std,weights=data_std['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']}),
              'PURPOSE':smf.wls(formula=topline_specs['PURPOSE'],data=data_std,weights=data_std['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']}),
              'RD_GO':smf.wls(formula=topline_specs['RD_GO'],data=data_std,weights=data_std['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']}),
              'RD_RV':smf.wls(formula=topline_specs['RD_RV'],data=data_std,weights=data_std['DENSITY']).\
                      fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']})}

topline_mods_std['GO'].summary()

NameError: name 'GO_PC_LN' is not defined

In [ ]:
topline_mods_std['GO'].summary()

In [ ]:
topline_mods_std['RV'].summary()

In [ ]:
topline_mods_std['ISSUER'].summary()

In [ ]:
topline_mods_std['PURPOSE'].summary()

In [ ]:
topline_mods_std['RD_GO'].summary()

In [ ]:
topline_mods_std['RD_RV'].summary()

Add in years

dummy for recessions

cluster by BEA region

implicit price deflator for state and local government services

try total volume as dependent and proportions

interest rates

## Capturing Results from All Specifications

In [ ]:
#Build dict to hold model output
mod_dict={}

#For each grp...
for grp in spec_dict.keys():
    #...update the dict with a model sub_dict
    sub_dict={'TYPE':[],'AGG':[],'HIRES':[]}
    mod_dict.update({grp:sub_dict})

#Walk down the spec dict...
for grp in spec_dict.keys():
    print '***',grp,'***'
    for dep in spec_dict[grp].keys():
        print '>',dep
        for mod in spec_dict[grp][dep].keys():
            print '>>',mod
            for fe in spec_dict[grp][dep][mod].keys():
                print '>>>',fe
                #...run each specification...
                tmp_mod=smf.ols(formula=spec_dict[grp][dep][mod][fe],data=data_std).\
                        fit(cov_type='cluster',cov_kwds={'groups':data_std['BEA_INT']})
                #...and deposit the model object in the appropriate mod_dict spot
                mod_dict[grp][mod].append(tmp_mod)

We have captured all the models.  First, is the explanatory power of our topline models representative?

In [ ]:
# dquery=data_d.reset_index()

print data_d.groupby(['Year']).count()['FIPS']
print data_d.groupby(['Year']).count()['FIPS'].mean()

In [ ]:
#Establish groups and model spec lists
groups=spec_dict.keys()
models=spec_dict['PC_GO']['Total'].keys()

#Generate plot object
fig,ax=plt.subplots(5,3,figsize=(18,25),sharex=True,sharey=True)

#For each group...
for i,grp in enumerate(groups):
    #...and each model spec...
    for j,model in enumerate(models):
        #...capture adjusted R squared from all models...
        r2_vec=np.array([mod.rsquared_adj for mod in mod_dict[grp][model]])
        #...and plot the histogram of adjust R squared
        ax[i,j].hist(r2_vec,color='#99000d',ec='w',bins=15)
        #...fix labels...
        ax[i,j].set_title('Group: '+grp+'; Spec: '+model)
        ax[i,j].set_xlabel('Adjusted $R^2$')
        ax[i,j].set_ylabel('Number of Models')
        #...fix axes
        simpleaxis(ax[i,j])
        
plt.suptitle('Distribution of Adjusted $R^2$ by Dependent Group and Base Specification',fontsize=20)

plt.savefig('../figures/r2_by_grp_spec.png',dpi=500,bbox_inches='tight')

In [ ]:
#Define dictionary to hold TEL variables by model
tel_dict={'TYPE':['TYPE1','TYPE2','TYPE2_Y'],
          'AGG':['LIMITS','BOTH'],
          'HIRES':['RATE_L','ASMT_L','GP_LMT','SC_LMT']}

#Establish list of colors
tel_cols=['#e41a1c','#377eb8','#4daf4a','#ff7f00']

def est_plot(dep_str,mod_str,data_highlight,ax=ax):
    #Create containers for density plots
    dplots=[]
    
    #For each TEL variable...
    for i,tvar in enumerate(tel_dict[mod_str]):
        #Capture data and dependent in tuples
        data_items=[(mod.params.ix[tvar],mod.model.endog_names) for mod in mod_dict[dep_str][mod_str]]
        #Capture the data in isolation
        data_vals=np.array([item[0] for item in data_items])
        #Capture data points associated with the dependents we wish to highlight
        data_hl=[item[0] for item in data_items if item[1]==data_highlight]
        #Plot density of coefficient estimates
        sb.distplot(data_vals,hist=False,rug=True,color=tel_cols[i],ax=ax)
        #Plot data highlight scatter
        p=ax.scatter(x=data_hl,y=[i+1 for val in data_hl],s=300,alpha=.5,c=tel_cols[i])
        #Capture plot
        dplots.append((p,tvar))
    #Generate zero line
    ax.axvline(x=0,color='k',linestyle='--')
    #Fix axes
    simpleaxis(ax)
    ax.legend([p[0] for p in dplots],[p[1] for p in dplots])
    
#Generate plot object
fig,ax=plt.subplots(figsize=(15,5))

#Plot per capita GO TEL estimates
est_plot('PC_GO','TYPE','GO_PC',ax=ax)

#Fix labels
ax.set_xlabel('Volume of GO Debt Issues Per Capita')
ax.set_ylabel('Number of Models')
ax.set_title('Impact of TELs on GO Debt by Capacity to Bind',fontsize=18)

#Fix axes
ax.set_xlim([-.15,.15])
ax.set_ylim([0,15])

plt.savefig('../figures/tel_on_go_debt_by_bind.png',dpi=500,bbox_inches='tight')

In [ ]:
#Generate plot object
fig,ax=plt.subplots(figsize=(15,5))

#Plot per capita GO TEL estimates
est_plot('PC_RV','TYPE','RV_PC',ax=ax)

#Fix labels
ax.set_xlabel('Volume of RV Debt Issues Per Capita')
ax.set_ylabel('Number of Models')
ax.set_title('Impact of TELs on RV Debt by Capacity to Bind',fontsize=18)

#Fix axes
ax.set_xlim([-.15,.15])
ax.set_ylim([0,15])

plt.savefig('../figures/tel_on_rv_debt_by_bind.png',dpi=500,bbox_inches='tight')

In [ ]:
#Generate plot object
fig,ax=plt.subplots(figsize=(15,5))

#Plot per capita GO TEL estimates
est_plot('PC_GO','AGG','GO_PC',ax=ax)

#Fix labels
ax.set_xlabel('Volume of GO Debt Issues Per Capita')
ax.set_ylabel('Number of Models')
ax.set_title('Impact of TEL Existence on GO Debt',fontsize=18)

#Fix axes
ax.set_xlim([-.15,.15])
ax.set_ylim([0,13])

plt.savefig('../figures/tel_on_go_debt_by_exist.png',dpi=500,bbox_inches='tight')

In [ ]:
#Generate plot object
fig,ax=plt.subplots(figsize=(15,5))

#Plot per capita GO TEL estimates
est_plot('PC_RV','AGG','RV_PC',ax=ax)

#Fix labels
ax.set_xlabel('Volume of RV Debt Issues Per Capita')
ax.set_ylabel('Number of Models')
ax.set_title('Impact of TEL Existence on RV Debt',fontsize=18)

#Fix axes
ax.set_xlim([-.15,.15])
ax.set_ylim([0,12])

plt.savefig('../figures/tel_on_rv_debt_by_exist.png',dpi=500,bbox_inches='tight')

In [ ]:
#Generate plot object
fig,ax=plt.subplots(figsize=(15,5))

#Plot per capita GO TEL estimates
est_plot('PC_GO','HIRES','GO_PC',ax=ax)

#Fix labels
ax.set_xlabel('Volume of GO Debt Issues Per Capita')
ax.set_ylabel('Number of Models')
ax.set_title('Impact of TELs on GO Debt by TEL Coverage',fontsize=18)

#Fix axes
ax.set_xlim([-.15,.15])
ax.set_ylim([0,10])

plt.savefig('../figures/tel_on_go_debt_by_cover.png',dpi=500,bbox_inches='tight')

In [ ]:
#Generate plot object
fig,ax=plt.subplots(figsize=(15,5))

#Plot per capita GO TEL estimates
est_plot('PC_RV','HIRES','RV_PC',ax=ax)

#Fix labels
ax.set_xlabel('Volume of RV Debt Issues Per Capita')
ax.set_ylabel('Number of Models')
ax.set_title('Impact of TELs on RV Debt by TEL Coverage',fontsize=18)

#Fix axes
ax.set_xlim([-.15,.15])
ax.set_ylim([0,10])

plt.savefig('../figures/tel_on_rv_debt_by_cover.png',dpi=500,bbox_inches='tight')